# planMe

## Configuration

In [465]:
BASD = "20110606"
ETS = "20220827"
DOR = [
    ("E-3", BASD),
    ("E-4", "20130801"),
    ("E-5", "20151101"),
    ("E-6", "20181101"),
    ("W-1", "20211101"),
    ("W-2", "20241101"),
    ("W-3", "20281101"),
]

BIRTHDAY = "19920821"

zipcode = "21144"
dependants = False

BRS = False

retirementYears = 20

# Format: (Name, Amount, Start date, End Date, Taxable),
AdditionalIncomes = [
    ("AIP", 1000, "20191101", "", False),
    ("SDAP", 150, "20191101", DOR[4][1], False),
    ("BAS", 375, BASD, DOR[4][1], False),
    ("BAS2", 258, DOR[4][1], "", False)
]

deductions = [
    ("Alimony", 1200, "20200401", "20210801"),
]

# ( amount, Years)
reenlistmentBonuses = [
    (82000, "20210801", 6),
]

## Data Injest

In [412]:
ranks = [
    "E-1",
    "E-2",
    "E-3",
    "E-4",
    "E-5",
    "E-6",
    "E-7",
    "E-8",
    "E-9",
    "W-1",
    "W-2",
    "W-3",
    "W-4",
    "W-5",
    "O-1E",
    "O-2E",
    "O-3E",
    "O-1",
    "O-2",
    "O-3",
    "O-4",
    "O-5",
    "O-6",
    "O-7"
]

In [413]:
withDependstr = """
AK400 KETCHIKAN, AK 1839 1839 1839 1839 2217 2232 2325 2454 2583 2235 2379 2535 2604 2694 2352 2511 2616 2226 2229 2526 2721 2856 2880 2901
AK401 SITKA, AK 1938 1938 1938 1938 2343 2358 2364 2400 2523 2361 2379 2421 2565 2739 2373 2412 2589 2352 2355 2418 2799 3075 3099 3123
AK402 JUNEAU, AK 1866 1866 1866 1866 2502 2517 2589 2697 2802 2520 2634 2763 2820 2889 2613 2745 2829 2511 2514 2757 2907 3012 3039 3060
AK403 KODIAK ISLAND, AK 1662 1662 1662 1662 2049 2064 2181 2334 2508 2067 2244 2430 2541 2676 2211 2403 2559 2058 2061 2421 2718 2928 2955 2976
AK404 ANCHORAGE, AK 1707 1707 1707 1707 2088 2103 2253 2442 2607 2106 2331 2562 2625 2703 2292 2526 2634 2097 2100 2553 2724 2841 2865 2886
AK405 FAIRBANKS, AK 1464 1464 1464 1464 1998 2013 2094 2205 2364 2016 2139 2277 2400 2550 2118 2256 2421 2007 2010 2271 2601 2835 2859 2880
AL001 ANNISTON/FORT MCCLELLAN, AL 933 933 933 933 975 1209 1227 1236 1269 1224 1233 1245 1290 1374 1230 1239 1299 1008 1206 1242 1398 1527 1539 1545
AL002 FORT RUCKER, AL 1062 1062 1062 1062 1110 1194 1350 1530 1698 1206 1425 1644 1722 1818 1389 1611 1734 1125 1191 1635 1848 1998 2013 2028
AL003 HUNTSVILLE, AL 1359 1359 1359 1359 1431 1713 1731 1740 1776 1728 1737 1749 1797 1884 1734 1743 1806 1470 1710 1746 1908 2043 2061 2073
AL004 MOBILE, AL 1389 1389 1389 1389 1500 1533 1641 1767 1902 1545 1692 1845 1926 2025 1665 1821 1938 1509 1530 1839 2058 2211 2229 2244
AL005 MONTGOMERY, AL 1191 1191 1191 1191 1245 1275 1326 1389 1485 1287 1353 1428 1512 1611 1338 1416 1524 1254 1272 1425 1644 1800 1815 1824
AL006 AUBURN, AL 1125 1125 1125 1125 1179 1314 1461 1626 1782 1326 1530 1731 1803 1896 1494 1701 1815 1200 1311 1722 1923 2061 2079 2091
AL007 BIRMINGHAM, AL 1461 1461 1461 1461 1596 1674 1689 1698 1731 1686 1695 1707 1749 1833 1692 1701 1761 1611 1671 1704 1860 1989 2004 2016
AR010 LITTLE ROCK, AR 1383 1383 1383 1383 1470 1518 1533 1548 1620 1530 1539 1563 1644 1746 1536 1557 1656 1482 1515 1560 1779 1935 1950 1962
AR012 FORT CHAFFEE/FORT SMITH, AR 855 855 855 855 897 1152 1242 1347 1434 1164 1287 1413 1446 1488 1266 1392 1449 930 1149 1407 1497 1557 1569 1578
AR411 FAYETTEVILLE, AR 1023 1023 1023 1023 1116 1329 1404 1488 1623 1344 1440 1542 1656 1797 1422 1527 1677 1146 1326 1536 1842 2061 2076 2091
AZ013 PHOENIX, AZ 1605 1605 1605 1605 1710 1929 1971 2025 2073 1944 1995 2055 2082 2115 1983 2046 2085 1743 1926 2052 2121 2169 2187 2199
AZ014 FORT HUACHUCA, AZ 1062 1062 1062 1062 1137 1425 1452 1482 1521 1440 1464 1503 1530 1569 1458 1494 1533 1176 1422 1500 1578 1632 1644 1653
AZ015 DAVIS-MONTHAN AFB, AZ 1308 1308 1308 1308 1446 1716 1734 1743 1767 1731 1740 1752 1782 1851 1737 1746 1791 1482 1713 1749 1872 1977 1992 2004
AZ016 YUMA, AZ 1041 1041 1041 1041 1116 1407 1443 1485 1554 1422 1461 1512 1572 1647 1452 1503 1581 1155 1404 1509 1668 1779 1794 1803
CA018 OAKLAND, CA 3228 3228 3228 3228 3366 3651 3690 3738 3900 3666 3711 3768 3951 4170 3702 3759 3981 3402 3648 3765 4245 4593 4635 4674
CA019 SAN FRANCISCO, CA 4536 4536 4536 4536 4971 5064 5079 5088 5202 5076 5085 5097 5250 5457 5082 5091 5280 4989 5061 5094 5529 5859 5913 5964
CA021 CHINA LAKE, CA 981 981 981 981 1032 1281 1419 1572 1725 1296 1482 1671 1749 1842 1449 1641 1761 1068 1278 1662 1872 2019 2037 2049
CA022 FRESNO, CA 1497 1497 1497 1497 1593 1767 1836 1920 2016 1779 1872 1971 2034 2112 1854 1956 2043 1620 1764 1968 2133 2250 2268 2283
CA023 LEMOORE NAS, CA 1476 1476 1476 1476 1518 1740 1758 1767 1785 1755 1764 1776 1803 1866 1761 1770 1809 1551 1737 1773 1884 1980 1995 2007
CA024 CAMP PENDLETON, CA 2574 2574 2574 2574 2793 3189 3210 3219 3270 3207 3216 3228 3297 3414 3213 3222 3312 2844 3186 3225 3450 3633 3663 3693
CA025 VENTURA, CA 2616 2616 2616 2616 2688 3003 3027 3054 3156 3018 3039 3075 3192 3333 3033 3066 3210 2730 3000 3072 3381 3606 3639 3666
CA026 VANDENBERG AFB, CA 2097 2097 2097 2097 2265 2388 2403 2412 2442 2400 2409 2421 2460 2535 2406 2415 2469 2286 2385 2418 2556 2673 2694 2712
CA027 MARIN/SONOMA, CA 3141 3141 3141 3141 3231 3726 3747 3756 3915 3744 3753 3765 3984 4272 3750 3759 4026 3291 3723 3762 4377 4839 4881 4923
CA028 BARSTOW/FORT IRWIN, CA 1311 1311 1311 1311 1359 1548 1614 1695 1788 1560 1650 1743 1806 1884 1632 1728 1815 1386 1545 1740 1908 2028 2043 2055
CA031 SAN BERNARDINO, CA 2226 2226 2226 2226 2430 2781 2799 2808 2820 2796 2805 2817 2823 2829 2802 2811 2826 2475 2778 2814 2832 2835 2838 2859
CA032 TWENTY NINE PALMS MCB, CA 1131 1131 1131 1131 1206 1416 1494 1584 1704 1431 1533 1638 1731 1842 1512 1623 1746 1236 1413 1635 1878 2049 2067 2079
CA033 BEALE AFB, CA 2013 2013 2013 2013 2112 2319 2334 2343 2403 2331 2340 2352 2433 2556 2337 2346 2448 2142 2316 2349 2598 2790 2814 2835
CA034 SACRAMENTO, CA 2112 2112 2112 2112 2286 2415 2430 2454 2523 2427 2442 2472 2544 2637 2436 2466 2556 2307 2412 2469 2667 2808 2832 2853
CA035 STOCKTON, CA 1686 1686 1686 1686 1713 1845 1887 1938 2079 1857 1908 1968 2121 2307 1896 1959 2148 1734 1842 1965 2370 2661 2682 2703
CA036 VALLEJO/TRAVIS AFB, CA 2532 2532 2532 2532 2727 2748 2763 2772 2802 2760 2769 2781 2823 2913 2766 2775 2835 2736 2745 2778 2940 3075 3102 3123
CA037 LOS ANGELES, CA 3006 3006 3006 3006 3354 3483 3498 3507 3609 3495 3504 3516 3651 3825 3501 3510 3675 3375 3480 3513 3885 4161 4197 4233
CA038 SAN DIEGO, CA 2691 2691 2691 2691 2949 3192 3225 3267 3366 3207 3243 3291 3396 3522 3234 3282 3414 2982 3189 3288 3564 3765 3798 3828
CA039 MONTEREY, CA 2418 2418 2418 2418 2499 3090 3339 3609 3861 3111 3453 3783 3891 4023 3396 3735 3909 2571 3084 3768 4068 4275 4311 4347
CA041 RIVERSIDE, CA 2091 2091 2091 2091 2199 2355 2370 2388 2457 2367 2376 2403 2481 2580 2373 2397 2493 2220 2352 2400 2610 2763 2787 2805
CA042 HUMBOLDT COUNTY, CA 1368 1368 1368 1368 1473 1788 1908 2043 2229 1803 1965 2130 2268 2439 1938 2103 2292 1512 1785 2121 2496 2760 2784 2805
CA044 SANTA CLARA COUNTY, CA 3849 3849 3849 3849 4119 4707 4806 4914 5211 4725 4851 4980 5298 5679 4830 4962 5355 4191 4701 4974 5817 6423 6483 6537
CA392 SAN LUIS OBISPO, CA 2073 2073 2073 2073 2226 2895 2934 2976 3078 2913 2952 3000 3111 3246 2943 2991 3129 2307 2889 2997 3291 3504 3534 3561
CA393 BRIDGEPORT, CA 1341 1341 1341 1341 1410 1965 1986 1995 2070 1983 1992 2004 2106 2259 1989 1998 2127 1479 1962 2001 2310 2550 2571 2589
CA420 ELCENTRO, CA 1320 1320 1320 1320 1380 1464 1491 1527 1596 1476 1506 1551 1617 1704 1497 1542 1629 1395 1461 1548 1728 1857 1872 1884
CA457 EDWARDS AFB/PALMDALE, CA 1857 1857 1857 1857 1911 2013 2142 2289 2433 2025 2205 2382 2457 2547 2172 2355 2469 1929 2010 2373 2574 2715 2739 2757
CO045 DENVER, CO 1947 1947 1947 1947 2169 2475 2493 2502 2526 2490 2499 2511 2544 2619 2496 2505 2553 2211 2472 2508 2637 2748 2772 2793
CO046 COLORADO SPRINGS, CO 1827 1827 1827 1827 1947 2004 2019 2031 2082 2016 2025 2043 2100 2172 2022 2037 2109 1959 2001 2040 2193 2304 2322 2340
CO047 FORT COLLINS, CO 1623 1623 1623 1623 1704 1860 1935 2025 2124 1872 1974 2079 2142 2220 1953 2064 2151 1728 1857 2076 2244 2361 2382 2397
CO422 BOULDER, CO 1989 1989 1989 1989 2172 2598 2643 2694 2766 2616 2664 2724 2784 2862 2655 2715 2793 2226 2595 2721 2883 3000 3024 3048
CT049 NEW LONDON, CT 1470 1470 1470 1470 1548 2022 2043 2052 2088 2040 2049 2061 2109 2196 2046 2055 2118 1608 2019 2058 2220 2352 2373 2388
CT050 HARTFORD, CT 1887 1887 1887 1887 1923 2166 2217 2280 2406 2181 2244 2316 2442 2598 2232 2304 2463 1956 2163 2313 2649 2889 2913 2937
CT051 NEW HAVEN/FAIRFIELD, CT 2754 2754 2754 2754 2757 2997 3015 3024 3081 3012 3021 3033 3111 3237 3018 3027 3126 2787 2994 3030 3276 3471 3501 3528
DC053 WASHINGTON, DC METRO AREA 2520 2520 2520 2520 2544 2700 2865 3051 3225 2715 2943 3168 3249 3351 2904 3135 3264 2568 2697 3159 3384 3540 3570 3600
DE054 DOVER AFB/REHOBOTH, DE 1605 1605 1605 1605 1668 1689 1776 1881 1989 1701 1821 1947 2007 2082 1797 1929 2016 1677 1686 1941 2103 2217 2235 2250
FL056 EGLIN AFB, FL 1647 1647 1647 1647 1665 1776 1878 1995 2118 1788 1926 2070 2136 2220 1902 2049 2148 1683 1773 2064 2244 2370 2388 2406
FL057 GAINESVILLE, FL 1449 1449 1449 1449 1452 1593 1707 1839 1989 1605 1761 1923 2016 2130 1734 1899 2031 1470 1590 1914 2166 2343 2361 2376
FL058 JACKSONVILLE, FL 1617 1617 1617 1617 1674 1809 1890 1983 2064 1821 1929 2043 2073 2115 1911 2025 2079 1695 1806 2037 2124 2184 2202 2217
FL059 PATRICK AFB, FL 1755 1755 1755 1755 1839 1872 1917 1977 2043 1884 1944 2013 2055 2109 1929 2001 2061 1848 1869 2010 2121 2202 2217 2232
FL061 MIAMI/FORT LAUDERDALE, FL 2337 2337 2337 2337 2352 2670 2727 2793 2937 2685 2757 2835 2976 3150 2742 2823 3003 2394 2667 2832 3210 3483 3513 3540
FL062 ORLANDO, FL 1881 1881 1881 1881 1926 2049 2064 2073 2088 2061 2070 2082 2103 2157 2067 2076 2109 1947 2046 2079 2172 2253 2271 2286
FL063 PANAMA CITY, FL 1713 1713 1713 1713 1791 1944 2046 2166 2295 1956 2097 2241 2316 2412 2073 2220 2328 1815 1941 2235 2439 2583 2604 2625
FL064 PENSACOLA, FL 1461 1461 1461 1461 1503 1716 1749 1791 1857 1728 1767 1818 1875 1944 1758 1809 1884 1533 1713 1815 1965 2070 2088 2100
FL065 TALLAHASSEE, FL 1473 1473 1473 1473 1476 1497 1632 1785 1977 1509 1695 1884 2013 2172 1665 1857 2034 1485 1494 1875 2223 2469 2490 2508
FL066 TAMPA, FL 1959 1959 1959 1959 2082 2157 2172 2181 2214 2169 2178 2190 2235 2322 2175 2184 2247 2094 2154 2187 2349 2484 2502 2520
FL067 WEST PALM BEACH, FL 2115 2115 2115 2115 2226 2262 2322 2397 2511 2274 2355 2442 2538 2658 2337 2427 2556 2235 2259 2439 2697 2880 2904 2928
FL068 OCALA, FL 1482 1482 1482 1482 1530 1557 1572 1581 1659 1569 1578 1590 1686 1806 1575 1584 1701 1539 1554 1587 1845 2031 2046 2058
FL069 FLORIDA KEYS, FL 2679 2679 2679 2679 2817 3348 3531 3732 3960 3369 3615 3858 3999 4173 3573 3822 4023 2883 3342 3846 4233 4506 4545 4584
FL070 VOLUSIA COUNTY, FL 1638 1638 1638 1638 1752 1782 1797 1818 1881 1794 1806 1833 1899 1983 1800 1827 1911 1761 1779 1830 2007 2136 2154 2169
FL423 FORT PIERCE, FL 1683 1683 1683 1683 1848 1869 1884 1893 1950 1881 1890 1905 1968 2049 1887 1899 1977 1857 1866 1902 2073 2193 2211 2226
FL424 FT MYERS BCH, FL 1620 1620 1620 1620 1689 1740 1809 1890 1962 1752 1842 1941 1974 2013 1824 1926 1977 1701 1737 1938 2022 2079 2094 2109
GA071 ATLANTA, GA 2124 2124 2124 2124 2259 2385 2400 2409 2463 2397 2406 2418 2490 2610 2403 2412 2505 2277 2382 2415 2649 2832 2856 2877
GA072 ALBANY, GA 1008 1008 1008 1008 1035 1065 1173 1302 1467 1077 1227 1380 1500 1647 1200 1359 1521 1044 1062 1374 1695 1923 1938 1950
GA073 FORT GORDON, GA 1428 1428 1428 1428 1443 1581 1602 1629 1713 1593 1614 1647 1740 1851 1608 1641 1752 1464 1578 1644 1887 2064 2079 2094
GA074 KINGS BAY/BRUNSWICK, GA 1323 1323 1323 1323 1377 1437 1452 1473 1557 1449 1461 1488 1584 1704 1455 1482 1599 1389 1434 1485 1740 1926 1941 1950
GA075 FORT BENNING, GA 1173 1173 1173 1173 1200 1554 1575 1584 1596 1572 1581 1593 1602 1647 1578 1587 1605 1245 1551 1590 1656 1719 1731 1740
GA076 ROBINS AFB, GA 1227 1227 1227 1227 1302 1323 1377 1443 1578 1335 1404 1485 1617 1776 1392 1473 1638 1311 1320 1482 1830 2082 2097 2112
GA077 SAVANNAH, GA 1467 1467 1467 1467 1557 1752 1767 1776 1806 1764 1773 1785 1824 1899 1770 1779 1833 1587 1749 1782 1920 2037 2052 2064
GA079 DAHLONEGA, GA 1473 1473 1473 1473 1485 2070 2091 2100 2112 2088 2097 2109 2115 2127 2094 2103 2118 1557 2064 2106 2130 2163 2181 2196
GA080 FORT STEWART, GA 1428 1428 1428 1428 1512 1533 1590 1665 1773 1545 1623 1710 1800 1908 1605 1695 1812 1521 1530 1707 1944 2112 2130 2142
GA081 MOODY AFB, GA 1056 1056 1056 1056 1080 1188 1275 1377 1497 1200 1317 1440 1521 1620 1296 1422 1533 1098 1185 1437 1653 1806 1818 1830
HI407 MAUI COUNTY, HI 2529 2529 2529 2529 2859 3390 3510 3642 3783 3408 3564 3723 3807 3909 3537 3699 3819 2925 3387 3717 3942 4101 4137 4170
HI408 HONOLULU COUNTY, HI 2823 2823 2823 2823 2991 3012 3189 3390 3588 3024 3273 3519 3618 3738 3231 3480 3633 3000 3009 3507 3777 3966 4002 4032
HI409 HAWAII COUNTY, HI 2139 2139 2139 2139 2223 2430 2448 2457 2499 2445 2454 2466 2514 2583 2451 2460 2523 2250 2427 2463 2601 2703 2724 2745
HI414 KAUAI COUNTY,HI 2433 2433 2433 2433 2742 2997 3015 3024 3069 3012 3021 3033 3096 3201 3018 3027 3108 2778 2994 3030 3234 3399 3429 3456
IA082 DES MOINES, IA 1266 1266 1266 1266 1386 1695 1758 1827 1926 1710 1785 1869 1947 2046 1770 1857 1959 1428 1692 1866 2073 2223 2241 2256
ID084 BOISE, ID 1422 1422 1422 1422 1431 1638 1668 1707 1782 1650 1686 1731 1803 1896 1677 1722 1815 1461 1635 1728 1923 2067 2082 2097
ID086 MOUNTAIN HOME AFB, ID 1323 1323 1323 1323 1500 1824 1845 1869 1932 1842 1854 1884 1950 2037 1851 1878 1962 1542 1821 1881 2064 2196 2211 2226
IL088 CHAMPAIGN/URBANA, IL 1032 1032 1032 1032 1089 1356 1374 1398 1485 1368 1383 1413 1515 1638 1380 1407 1530 1125 1353 1410 1680 1875 1887 1899
IL089 ROCK ISLAND, IL 1530 1530 1530 1530 1647 1764 1875 2004 2187 1776 1929 2085 2229 2406 1902 2061 2253 1665 1761 2076 2466 2745 2766 2787
IL090 PEORIA, IL 1125 1125 1125 1125 1182 1710 1734 1758 1836 1728 1746 1776 1863 1971 1740 1767 1875 1245 1707 1773 2004 2172 2190 2205
IL092 GREAT LAKES NAVTRACEN, IL 1530 1530 1530 1530 1599 2148 2244 2346 2511 2166 2286 2412 2550 2718 2265 2394 2574 1665 2142 2406 2772 3036 3063 3084
IL093 SCOTT AFB, IL 1161 1161 1161 1161 1221 1395 1482 1581 1719 1407 1524 1644 1752 1884 1503 1626 1770 1248 1392 1638 1926 2133 2148 2163
IL325 CHICAGO, IL 1920 1920 1920 1920 2058 2277 2298 2322 2400 2292 2307 2340 2424 2529 2304 2334 2436 2091 2274 2337 2562 2724 2748 2766
IL335 SPRINGFIELD/DECATUR,IL 921 921 921 921 987 1509 1530 1539 1551 1527 1536 1548 1563 1611 1533 1542 1566 1050 1506 1545 1620 1692 1704 1713
IN094 INDIANAPOLIS, IN 1443 1443 1443 1443 1506 1527 1545 1575 1626 1539 1557 1593 1644 1707 1551 1584 1650 1515 1524 1590 1725 1824 1836 1848
IN097 FORT WAYNE, IN 1323 1323 1323 1323 1530 1551 1593 1650 1764 1563 1617 1683 1797 1935 1605 1674 1815 1539 1548 1680 1980 2196 2214 2229
IN338 TERRE HAUTE, IN 951 951 951 951 975 1011 1026 1035 1113 1023 1032 1044 1143 1272 1029 1038 1161 984 1008 1041 1314 1515 1524 1533
IN399 BLOOMINGTON, IN 1170 1170 1170 1170 1257 1380 1497 1632 1839 1395 1554 1716 1887 2091 1527 1692 1914 1275 1377 1710 2163 2484 2505 2523
KS100 FORT RILEY, KS 1047 1047 1047 1047 1113 1308 1365 1431 1518 1320 1392 1473 1536 1617 1380 1461 1548 1143 1305 1470 1641 1761 1776 1785
KS101 WICHITA/MCCONNELL AFB, KS 993 993 993 993 1011 1362 1383 1392 1452 1380 1389 1401 1482 1614 1386 1395 1500 1056 1359 1398 1656 1860 1875 1887
KS102 FORT LEAVENWORTH, KS 1494 1494 1494 1494 1590 1656 1671 1686 1734 1668 1677 1701 1752 1818 1674 1695 1758 1602 1653 1698 1833 1932 1947 1959
KS105 TOPEKA, KS 1038 1038 1038 1038 1167 1440 1458 1467 1506 1455 1464 1476 1527 1617 1461 1470 1539 1203 1437 1473 1641 1776 1791 1800
KY106 FORT CAMPBELL, KY 1356 1356 1356 1356 1380 1500 1599 1716 1848 1512 1647 1788 1872 1974 1623 1767 1884 1401 1497 1782 2004 2163 2181 2196
KY107 LEXINGTON, KY 1227 1227 1227 1227 1299 1410 1425 1434 1506 1422 1431 1443 1533 1644 1428 1437 1548 1317 1407 1440 1680 1851 1866 1878
KY109 LOUISVILLE, KY 1440 1440 1440 1440 1566 1653 1731 1821 1917 1665 1767 1878 1935 2007 1749 1860 1944 1581 1650 1872 2028 2139 2157 2169
KY110 FORT KNOX, KY 1002 1002 1002 1002 1047 1374 1458 1554 1665 1389 1500 1614 1689 1779 1479 1596 1698 1089 1371 1608 1809 1950 1965 1977
KY339 FRANKFORT, KY 882 882 882 882 903 1101 1209 1332 1485 1116 1263 1410 1515 1644 1236 1389 1533 933 1098 1404 1686 1884 1899 1911
KY430 PADUCAH, KY 951 951 951 951 957 1176 1230 1296 1401 1188 1257 1335 1428 1545 1245 1323 1443 987 1173 1332 1581 1758 1773 1782
LA113 ALEXANDRIA, LA 879 879 879 879 897 1290 1311 1335 1479 1308 1323 1350 1530 1749 1317 1344 1560 945 1287 1347 1824 2169 2187 2202
LA114 BATON ROUGE, LA 1428 1428 1428 1428 1539 1755 1824 1905 2028 1770 1860 1956 2058 2184 1842 1941 2073 1569 1752 1950 2223 2418 2439 2454
LA115 FORT POLK, LA 876 876 876 876 987 1302 1332 1368 1431 1317 1347 1389 1452 1527 1341 1383 1461 1029 1299 1386 1548 1665 1677 1686
LA116 NEW ORLEANS, LA 1455 1455 1455 1455 1647 1818 1893 1977 2124 1830 1929 2031 2163 2322 1911 2016 2184 1671 1815 2025 2376 2625 2649 2667
LA117 SHREVEPORT/BARKSDALE AFB, LA 1326 1326 1326 1326 1428 1806 1827 1836 1869 1824 1833 1845 1890 1977 1830 1839 1902 1476 1803 1842 2004 2136 2154 2169
LA118 LAFAYETTE, LA 1326 1326 1326 1326 1395 1671 1689 1698 1770 1686 1695 1707 1803 1944 1692 1701 1821 1434 1668 1704 1989 2208 2226 2241
LA326 ST MARY AND TERREBONNE, LA 1203 1203 1203 1203 1305 1398 1452 1521 1620 1410 1482 1566 1644 1743 1467 1551 1656 1320 1395 1560 1770 1923 1938 1947
LA370 LAKE CHARLES, LA 1392 1392 1392 1392 1500 1977 1998 2007 2019 1995 2004 2016 2022 2031 2001 2010 2025 1557 1971 2013 2034 2064 2079 2094
LA371 MONROE, LA 918 918 918 918 951 1293 1332 1374 1461 1308 1350 1404 1485 1584 1341 1395 1497 996 1290 1401 1614 1770 1785 1794
MA119 NANTUCKET, MA 3156 3156 3156 3156 3369 3798 4002 4224 4437 3819 4095 4365 4464 4587 4047 4326 4482 3423 3795 4353 4629 4818 4863 4902
MA120 BOSTON, MA 2910 2910 2910 2910 3075 3513 3597 3687 3945 3531 3636 3744 4023 4356 3615 3726 4071 3129 3510 3738 4476 5007 5055 5097
MA122 WORCESTER, MA 2040 2040 2040 2040 2049 2631 2664 2700 2844 2649 2679 2724 2892 3096 2673 2715 2919 2121 2625 2721 3165 3486 3516 3543
MA123 FITCHBURG, MA 2094 2094 2094 2094 2244 2697 2718 2742 2805 2715 2730 2760 2826 2913 2724 2751 2838 2301 2694 2757 2937 3069 3093 3117
MA124 CAPE COD-PLYMOUTH, MA 2370 2370 2370 2370 2400 2673 2748 2832 2973 2688 2784 2886 3006 3153 2766 2871 3027 2436 2670 2883 3201 3429 3459 3486
MA125 ESSEX CO, MA 2328 2328 2328 2328 2502 2829 2856 2889 3051 2844 2871 2910 3108 3345 2865 2904 3141 2544 2826 2907 3429 3804 3837 3870
MA126 HAMPDEN COUNTY, MA 1563 1563 1563 1563 1650 2034 2121 2214 2364 2052 2160 2271 2400 2553 2139 2256 2421 1698 2031 2268 2604 2847 2871 2892
MA151 MARTHA'S VINEYARD, MA 2448 2448 2448 2448 2748 3084 3186 3300 3420 3102 3234 3372 3438 3525 3210 3351 3450 2793 3081 3366 3549 3678 3711 3738
MA377 HANSCOM AFB, MA 2598 2598 2598 2598 2694 3132 3222 3321 3546 3150 3264 3384 3609 3876 3243 3366 3648 2748 3129 3378 3972 4398 4437 4476
MD127 ABERDEEN PROVING GROUNDS, MD 1713 1713 1713 1713 1821 1998 2079 2172 2307 2010 2118 2232 2337 2463 2100 2214 2352 1848 1995 2226 2502 2700 2724 2742
MD128 ANNAPOLIS, MD 2190 2190 2190 2190 2283 2538 2556 2565 2646 2553 2562 2574 2685 2844 2559 2568 2706 2316 2535 2571 2898 3147 3174 3198
MD129 BALTIMORE, MD 1911 1911 1911 1911 2070 2328 2373 2427 2553 2343 2397 2460 2589 2748 2385 2451 2613 2106 2325 2457 2799 3045 3072 3096
MD130 FORT DETRICK, MD 1734 1734 1734 1734 1824 2097 2115 2124 2199 2112 2121 2133 2232 2373 2118 2127 2253 1860 2094 2130 2421 2643 2664 2682
MD133 FORT G. G. MEADE, MD 2100 2100 2100 2100 2190 2502 2538 2577 2673 2517 2556 2604 2703 2823 2547 2595 2718 2229 2499 2601 2862 3051 3078 3102
MD134 INDIAN HEAD NAVORDSTA, MD 2160 2160 2160 2160 2337 2454 2469 2484 2607 2466 2475 2499 2649 2835 2472 2493 2676 2355 2451 2496 2898 3189 3216 3240
MD135 PATUXENT RIVER, MD 1692 1692 1692 1692 1791 2106 2124 2133 2175 2121 2130 2142 2196 2295 2127 2136 2208 1830 2103 2139 2322 2472 2493 2511
MD432 OCEAN CITY, MD 1332 1332 1332 1332 1338 1590 1620 1659 1743 1605 1638 1683 1770 1881 1629 1674 1785 1371 1587 1680 1914 2085 2100 2115
MD458 OXFORD,MD 1686 1686 1686 1686 1902 2238 2256 2265 2286 2253 2262 2274 2301 2367 2259 2268 2310 1947 2235 2271 2385 2484 2502 2520
ME136 BRUNSWICK, ME 1368 1368 1368 1368 1629 1728 2157 2628 2991 1749 2352 2928 3015 3120 2253 2844 3027 1647 1725 2904 3153 3315 3345 3369
ME139 PORTLAND, ME 2052 2052 2052 2052 2055 2583 2622 2664 2817 2601 2640 2691 2868 3078 2631 2682 2898 2115 2577 2688 3153 3489 3519 3546
ME141 COASTAL MAINE, ME 1467 1467 1467 1467 1719 1887 1902 1923 2010 1899 1911 1938 2037 2163 1905 1932 2055 1743 1884 1935 2202 2397 2415 2433
ME390 BANGOR, ME 1155 1155 1155 1155 1344 1650 1668 1677 1695 1665 1674 1686 1704 1746 1671 1680 1710 1386 1647 1683 1752 1809 1824 1833
MI142 DETROIT, MI 1737 1737 1737 1737 1965 2268 2286 2295 2403 2283 2292 2304 2454 2661 2289 2298 2481 2004 2265 2301 2733 3063 3087 3111
MI143 MARQUETTE, MI 1191 1191 1191 1191 1212 1644 1689 1740 1818 1662 1713 1770 1836 1917 1701 1761 1845 1266 1641 1767 1941 2064 2079 2094
MI145 SAULT STE MARIE, MI 1047 1047 1047 1047 1164 1191 1206 1215 1245 1203 1212 1224 1266 1344 1209 1218 1275 1173 1188 1221 1365 1485 1494 1503
MI146 TRAVERSE CITY, MI 1524 1524 1524 1524 1674 1698 1797 1917 2031 1710 1848 1992 2049 2121 1824 1971 2058 1683 1695 1986 2139 2247 2265 2280
MI148 GRAND HAVEN, MI 1416 1416 1416 1416 1488 1758 1776 1797 1836 1773 1785 1812 1851 1902 1782 1806 1854 1524 1755 1809 1914 1989 2004 2016
MI152 BATTLE CREEK/KALAMAZOO, MI 1140 1140 1140 1140 1263 1656 1689 1731 1842 1671 1707 1755 1875 2025 1698 1746 1896 1314 1653 1752 2073 2307 2325 2340
MI153 LANSING, MI 1305 1305 1305 1305 1446 1710 1743 1785 1869 1722 1761 1812 1893 1995 1752 1803 1908 1482 1707 1809 2025 2181 2199 2214
MI154 GRAND RAPIDS, MI 1479 1479 1479 1479 1659 1680 1695 1704 1791 1692 1701 1713 1824 1959 1698 1707 1842 1668 1677 1710 2004 2220 2238 2250
MI155 ANN ARBOR, MI 1776 1776 1776 1776 1947 2340 2394 2451 2532 2358 2418 2487 2550 2631 2406 2475 2562 1998 2337 2484 2655 2778 2799 2820
MI156 SAGINAW, MI 1011 1011 1011 1011 1146 1275 1290 1299 1311 1287 1296 1308 1320 1365 1293 1302 1326 1167 1272 1305 1374 1437 1446 1452
MN158 DULUTH, MN 1545 1545 1545 1545 1668 1755 1788 1833 1902 1767 1806 1860 1920 1998 1797 1851 1932 1686 1752 1857 2022 2139 2157 2169
MN159 MINNEAPOLIS/ST PAUL, MN 1617 1617 1617 1617 1734 2058 2103 2154 2229 2076 2127 2187 2247 2325 2115 2175 2256 1776 2055 2184 2346 2463 2481 2499
MO160 KANSAS CITY, MO 1512 1512 1512 1512 1563 1602 1617 1626 1653 1614 1623 1635 1671 1752 1620 1629 1683 1575 1599 1632 1776 1899 1911 1923
MO161 ST.LOUIS, MO 1491 1491 1491 1491 1494 1962 1983 1992 2034 1980 1989 2001 2058 2145 1986 1995 2067 1551 1959 1998 2172 2310 2331 2346
MO162 WHITEMAN AFB, MO 981 981 981 981 1086 1281 1383 1503 1641 1293 1434 1578 1668 1776 1410 1557 1683 1116 1278 1572 1812 1980 1995 2010
MO163 FORT LEONARD WOOD, MO 888 888 888 888 954 1080 1152 1236 1314 1092 1188 1290 1326 1374 1170 1272 1332 975 1077 1284 1386 1458 1467 1473
MO164 SPRINGFIELD, MO 966 966 966 966 1017 1140 1194 1257 1383 1152 1221 1296 1419 1566 1206 1284 1437 1038 1137 1293 1614 1848 1860 1872
MO165 COLUMBIA/JEFFERSON CITY,MO 918 918 918 918 972 1050 1278 1533 1737 1065 1383 1695 1752 1824 1332 1650 1761 987 1047 1683 1845 1953 1968 1980
MO344 SAINT JOSEPH, MO 957 957 957 957 1044 1314 1332 1341 1362 1329 1338 1350 1377 1443 1335 1344 1386 1080 1311 1347 1461 1560 1572 1581
MS168 GULFPORT, MS 1218 1218 1218 1218 1311 1341 1395 1461 1548 1353 1422 1500 1569 1650 1407 1488 1578 1320 1338 1497 1674 1803 1815 1827
MS169 COLUMBUS AFB, MS 936 936 936 936 939 1272 1329 1395 1512 1287 1356 1434 1542 1677 1344 1422 1560 981 1269 1431 1719 1929 1944 1953
MS170 JACKSON, MS 1383 1383 1383 1383 1494 1674 1704 1740 1839 1686 1719 1764 1869 1995 1710 1755 1884 1521 1671 1761 2037 2238 2256 2271
MS171 MERIDIAN, MS 939 939 939 939 996 1209 1305 1413 1566 1224 1350 1482 1599 1737 1329 1464 1617 1026 1206 1476 1782 2001 2016 2031
MS172 HATTIESBURG, MS 1164 1164 1164 1164 1200 1320 1392 1476 1602 1332 1425 1527 1629 1758 1410 1512 1647 1221 1317 1524 1797 1995 2010 2022
MT175 MALMSTROM AFB/GREAT FLS, MT 936 936 936 936 996 1158 1278 1416 1536 1170 1335 1503 1548 1608 1308 1479 1557 1020 1155 1497 1623 1707 1722 1731
MT347 HELENA, MT 987 987 987 987 1065 1470 1530 1596 1692 1485 1557 1638 1713 1809 1542 1626 1725 1116 1467 1632 1839 1986 2001 2013
NC176 OUTER BANKS, NC 1674 1674 1674 1674 1755 1947 1980 2019 2103 1962 1998 2046 2130 2232 1989 2037 2142 1785 1944 2043 2265 2427 2445 2463
NC177 MOREHEAD/CHERRY PT MCAS, NC 1305 1305 1305 1305 1308 1329 1383 1449 1554 1341 1410 1488 1581 1692 1395 1476 1593 1314 1326 1485 1725 1899 1914 1926
NC178 CAMP LEJEUNE, NC 1263 1263 1263 1263 1266 1287 1383 1497 1587 1299 1431 1569 1593 1629 1407 1548 1596 1272 1284 1563 1635 1686 1698 1707
NC179 CHARLOTTE, NC 1575 1575 1575 1575 1668 1764 1779 1788 1848 1776 1785 1797 1878 2004 1782 1791 1893 1686 1761 1794 2046 2244 2262 2277
NC180 DURHAM/CHAPEL HILL, NC 1533 1533 1533 1533 1632 1995 2016 2028 2112 2013 2022 2040 2145 2274 2019 2034 2160 1677 1992 2037 2316 2517 2538 2553
NC181 ELIZABETH CITY, NC 1512 1512 1512 1512 1527 1983 2004 2013 2055 2001 2010 2022 2082 2181 2007 2016 2094 1584 1980 2019 2214 2370 2391 2406
NC182 FORT BRAGG/POPE, NC 1233 1233 1233 1233 1314 1653 1671 1680 1701 1668 1677 1689 1716 1779 1674 1683 1722 1359 1650 1686 1794 1890 1905 1917
NC183 SEYMOUR JOHNSON AFB, NC 1101 1101 1101 1101 1104 1194 1323 1470 1608 1206 1383 1563 1629 1713 1353 1536 1641 1116 1191 1554 1737 1866 1881 1890
NC184 GREENSBORO, NC 1200 1200 1200 1200 1236 1485 1572 1671 1767 1497 1614 1734 1782 1839 1593 1716 1788 1272 1482 1731 1854 1944 1959 1971
NC185 RALEIGH, NC 1506 1506 1506 1506 1587 1857 1932 2013 2085 1872 1965 2067 2094 2130 1947 2052 2097 1623 1854 2061 2136 2190 2205 2220
NC186 WILMINGTON, NC 1368 1368 1368 1368 1398 1752 1770 1779 1833 1767 1776 1788 1863 1983 1773 1782 1878 1443 1749 1785 2019 2205 2223 2238
NC187 ASHEVILLE, NC 1479 1479 1479 1479 1596 1806 1848 1899 2001 1821 1869 1929 2031 2157 1860 1920 2049 1626 1803 1926 2196 2391 2409 2427
ND188 BISMARCK, ND 1170 1170 1170 1170 1266 1656 1707 1764 1854 1671 1731 1800 1875 1965 1719 1791 1887 1317 1653 1797 1992 2133 2148 2163
ND189 FARGO, ND 1065 1065 1065 1065 1191 1611 1632 1641 1653 1629 1638 1650 1659 1701 1635 1644 1662 1242 1608 1647 1710 1773 1788 1797
ND190 GRAND FORKS, ND 1224 1224 1224 1224 1353 1533 1548 1569 1629 1545 1557 1581 1650 1737 1551 1575 1659 1377 1530 1578 1761 1893 1908 1920
ND191 MINOT AFB, ND 1125 1125 1125 1125 1230 1716 1737 1746 1770 1734 1743 1755 1788 1857 1740 1749 1794 1290 1713 1752 1875 1983 1998 2010
NE192 OMAHA/OFFUTT AFB, NE 1254 1254 1254 1254 1428 1767 1788 1815 1896 1782 1800 1833 1923 2037 1794 1824 1938 1473 1764 1830 2070 2247 2265 2280
NE193 LINCOLN, NE 1068 1068 1068 1068 1119 1533 1551 1560 1614 1548 1557 1569 1632 1716 1554 1563 1644 1170 1530 1566 1740 1869 1884 1896
NH194 PORTSMOUTH, NH/KITTERY, ME 2016 2016 2016 2016 2253 2466 2673 2901 3129 2484 2766 3045 3159 3300 2721 3003 3180 2283 2463 3033 3345 3564 3594 3621
NH195 MANCHESTER/CONCORD, NH 2028 2028 2028 2028 2190 2553 2640 2736 2892 2568 2679 2796 2931 3093 2661 2778 2952 2235 2550 2790 3147 3405 3432 3459
NJ196 ATLANTIC CITY, NJ 1821 1821 1821 1821 1899 2346 2406 2469 2568 2364 2433 2511 2595 2697 2418 2499 2607 1956 2343 2508 2730 2889 2913 2934
NJ198 CAPE MAY, NJ 1575 1575 1575 1575 1701 1872 1953 2046 2139 1884 1992 2106 2154 2214 1971 2088 2160 1725 1869 2100 2229 2322 2340 2355
NJ200 FORT MONMOUTH/EARLE NWS, NJ 2376 2376 2376 2376 2433 2643 2766 2904 3117 2658 2823 2991 3168 3381 2793 2964 3198 2463 2640 2982 3456 3795 3828 3858
NJ201 PERTH AMBOY, NJ 2091 2091 2091 2091 2220 2658 2694 2733 2826 2676 2709 2757 2853 2973 2703 2748 2868 2277 2655 2754 3012 3198 3225 3249
NJ202 NORTHERN NEW JERSEY 2814 2814 2814 2814 3006 3219 3270 3333 3459 3231 3297 3372 3495 3642 3285 3360 3516 3036 3216 3369 3690 3924 3957 3990
NJ203 TRENTON, NJ 2172 2172 2172 2172 2400 2625 2661 2709 2823 2637 2682 2736 2856 3003 2673 2727 2877 2430 2622 2733 3054 3285 3312 3339
NJ204 JB MCGUIRE-DIX-LAKEHURST, NJ 1869 1869 1869 1869 1962 2280 2337 2403 2484 2295 2367 2445 2502 2571 2352 2433 2508 2004 2277 2439 2592 2700 2721 2742
NM205 HOLLOMAN AFB/ALAMOGORDO, NM 1092 1092 1092 1092 1230 1446 1473 1509 1569 1458 1491 1533 1584 1650 1482 1524 1593 1260 1443 1530 1668 1767 1779 1788
NM206 ALBUQUERQUE/KIRTLAND AFB, NM 1455 1455 1455 1455 1545 1572 1587 1596 1623 1584 1593 1605 1641 1722 1590 1599 1653 1554 1569 1602 1746 1869 1884 1893
NM207 CANNON AFB/CLOVIS, NM 924 924 924 924 1020 1299 1356 1422 1512 1314 1383 1461 1533 1620 1371 1449 1542 1056 1296 1458 1647 1779 1794 1803
NM209 WHITE SANDS MR/LAS CRUCES, NM 1041 1041 1041 1041 1224 1320 1335 1344 1377 1332 1341 1353 1389 1440 1338 1347 1395 1242 1317 1350 1452 1527 1539 1545
NM210 SANTA FE/LOS ALAMOS, NM 1626 1626 1626 1626 1758 1950 2139 2355 2556 1965 2229 2490 2583 2700 2184 2451 2601 1785 1947 2478 2739 2919 2943 2964
NV211 FALLON NAS, NV 1179 1179 1179 1179 1320 1425 1569 1734 1860 1437 1638 1836 1872 1920 1602 1806 1878 1338 1422 1827 1932 2001 2016 2031
NV212 NELLIS AFB/LAS VEGAS, NV 1587 1587 1587 1587 1674 1758 1779 1809 1872 1770 1794 1830 1890 1968 1785 1821 1899 1689 1755 1827 1989 2106 2124 2136
NV213 RENO/CARSON CITY, NV 1779 1779 1779 1779 1857 2103 2121 2130 2190 2118 2127 2139 2211 2310 2124 2133 2226 1890 2100 2136 2340 2493 2514 2532
NY215 BALLSTON SPA/ALBANY, NY 2040 2040 2040 2040 2145 2202 2256 2325 2442 2214 2286 2367 2472 2598 2271 2355 2487 2157 2199 2364 2637 2835 2859 2880
NY216 BUFFALO, NY 1875 1875 1875 1875 2070 2094 2139 2196 2286 2106 2163 2232 2310 2403 2151 2223 2322 2079 2091 2229 2433 2580 2601 2619
NY217 WEST POINT, NY 2118 2118 2118 2118 2157 2790 2811 2829 2940 2808 2817 2841 2979 3147 2814 2835 3003 2232 2784 2838 3204 3468 3498 3525
NY218 LONG ISLAND, NY 3357 3357 3357 3357 3594 3912 4026 4152 4353 3930 4080 4233 4401 4605 4053 4209 4431 3636 3909 4227 4674 4995 5040 5082
NY219 NEW YORK CITY, NY 3093 3093 3093 3093 3294 3477 3735 4023 4290 3495 3855 4206 4326 4470 3795 4152 4344 3321 3474 4191 4518 4746 4791 4830
NY221 ROCHESTER, NY 1440 1440 1440 1440 1617 1776 1809 1851 1941 1788 1827 1878 1965 2076 1818 1869 1980 1641 1773 1875 2112 2283 2301 2316
NY222 ROME/GRIFFISS AFB, NY 1386 1386 1386 1386 1617 2067 2136 2211 2280 2085 2166 2256 2292 2340 2151 2244 2298 1671 2064 2253 2349 2415 2436 2451
NY223 SYRACUSE, NY 1434 1434 1434 1434 1689 1779 1863 1965 2115 1791 1908 2028 2151 2298 1887 2010 2172 1704 1776 2022 2349 2583 2604 2622
NY225 FORT DRUM/WATERTOWN, NY 1245 1245 1245 1245 1326 1509 1731 1977 2157 1524 1833 2136 2166 2205 1782 2091 2169 1353 1506 2121 2214 2271 2289 2304
NY349 WESTCHESTER COUNTY, NY 3312 3312 3312 3312 3342 4368 4389 4398 4476 4386 4395 4407 4512 4668 4392 4401 4533 3462 4353 4404 4722 4968 5013 5055
NY413 STATEN ISLAND, NY 2895 2895 2895 2895 3024 3045 3249 3477 3720 3057 3342 3621 3762 3930 3294 3579 3783 3033 3042 3609 3987 4251 4287 4323
OH227 AKRON, OH 1188 1188 1188 1188 1224 1704 1728 1758 1890 1722 1743 1776 1935 2127 1734 1767 1962 1284 1698 1773 2193 2499 2520 2538
OH228 CINCINNATI, OH 1647 1647 1647 1647 1743 2040 2058 2067 2091 2055 2064 2076 2109 2178 2061 2070 2115 1782 2037 2073 2199 2304 2322 2337
OH229 CLEVELAND, OH 1392 1392 1392 1392 1413 1671 1698 1734 1833 1686 1713 1755 1863 1998 1707 1749 1881 1449 1668 1752 2040 2247 2265 2280
OH230 COLUMBUS, OH 1314 1314 1314 1314 1392 1722 1791 1872 1938 1737 1827 1923 1947 1980 1809 1908 1950 1434 1719 1917 1986 2031 2046 2061
OH231 WRIGHT-PATTERSON AFB, OH 1131 1131 1131 1131 1182 1443 1488 1539 1647 1458 1509 1572 1677 1803 1500 1563 1695 1218 1440 1569 1845 2043 2061 2073
OH232 TOLEDO, OH 1443 1443 1443 1443 1488 1614 1629 1638 1767 1626 1635 1647 1818 2034 1632 1641 1848 1509 1611 1644 2109 2454 2475 2493
OH233 YOUNGSTOWN, OH 924 924 924 924 1065 1188 1212 1245 1293 1200 1227 1263 1305 1359 1218 1257 1311 1086 1185 1260 1371 1449 1458 1467
OK235 ALTUS AFB, OK 885 885 885 885 987 1143 1224 1317 1434 1155 1263 1377 1458 1560 1245 1359 1473 1011 1140 1371 1593 1749 1761 1770
OK236 VANCE AFB/ENID, OK 969 969 969 969 1083 1254 1311 1383 1536 1266 1341 1425 1578 1761 1326 1413 1605 1107 1251 1422 1824 2115 2130 2145
OK237 FORT SILL/LAWTON, OK 885 885 885 885 954 1212 1254 1302 1395 1227 1275 1332 1422 1527 1263 1323 1434 990 1209 1329 1563 1728 1740 1749
OK239 OKLAHOMA CITY, OK 1206 1206 1206 1206 1308 1545 1566 1596 1677 1560 1581 1614 1704 1818 1572 1605 1719 1341 1542 1611 1854 2031 2046 2061
OK240 TULSA, OK 1119 1119 1119 1119 1164 1257 1299 1353 1482 1269 1323 1386 1521 1683 1311 1377 1545 1179 1254 1383 1737 1995 2010 2022
OR241 ASTORIA, OR 1392 1392 1392 1392 1542 1854 1896 1944 2034 1869 1917 1974 2058 2160 1905 1965 2070 1581 1851 1971 2190 2349 2370 2385
OR242 COOS BAY, OR 1356 1356 1356 1356 1461 1803 1902 2010 2133 1821 1947 2076 2154 2250 1923 2058 2166 1506 1800 2070 2280 2427 2448 2463
OR243 PORTLAND, OR 2427 2427 2427 2427 2430 2454 2493 2544 2667 2466 2514 2577 2703 2856 2502 2565 2724 2439 2451 2574 2907 3150 3177 3201
OR244 SALEM, OR 1428 1428 1428 1428 1521 1788 1839 1902 2001 1800 1866 1941 2028 2133 1854 1929 2040 1557 1785 1938 2166 2331 2352 2367
OR245 CORVALLIS, OR 1599 1599 1599 1599 1788 2178 2253 2337 2538 2196 2289 2388 2598 2847 2271 2373 2634 1839 2175 2385 2934 3330 3360 3387
OR246 EUGENE, OR 1761 1761 1761 1761 1866 1965 2004 2055 2178 1977 2028 2088 2214 2367 2016 2076 2235 1881 1962 2085 2418 2661 2682 2700
PA247 CARLISLE BARRACKS, PA 1482 1482 1482 1482 1599 1827 1890 1959 2109 1842 1920 2004 2151 2331 1905 1992 2178 1632 1824 2001 2394 2676 2697 2718
PA248 PHILADELPHIA, PA/CAMDEN, NJ 1848 1848 1848 1848 2001 2304 2322 2334 2412 2319 2328 2346 2436 2550 2325 2340 2451 2043 2301 2343 2586 2763 2784 2805
PA249 WILLOW GROVE, PA 1986 1986 1986 1986 2175 2475 2505 2538 2622 2490 2520 2559 2649 2757 2511 2553 2661 2217 2472 2556 2790 2961 2985 3006
PA250 PITTSBURGH, PA 1734 1734 1734 1734 1911 2262 2280 2289 2358 2277 2286 2298 2391 2535 2283 2292 2412 1956 2259 2295 2583 2808 2832 2853
PA252 STATE COLLEGE, PA 1287 1287 1287 1287 1431 1749 1773 1803 1896 1767 1788 1821 1926 2061 1779 1812 1944 1473 1746 1818 2103 2310 2328 2343
PA253 ERIE, PA 1242 1242 1242 1242 1299 1554 1572 1581 1602 1569 1578 1590 1611 1653 1575 1584 1617 1335 1551 1587 1662 1722 1734 1743
PA254 WILKES-BARRE/SCRANTON, PA 1374 1374 1374 1374 1533 1884 1902 1911 1950 1899 1908 1920 1974 2067 1905 1914 1986 1578 1881 1917 2097 2244 2262 2277
PA255 ALLENTOWN/BETHLEHEM, PA 1839 1839 1839 1839 1917 2052 2079 2115 2217 2064 2094 2139 2250 2391 2085 2130 2271 1938 2049 2136 2436 2652 2673 2694
RI256 NEWPORT, RI 1935 1935 1935 1935 2160 2667 2712 2760 2892 2685 2733 2790 2931 3102 2721 2781 2955 2223 2664 2787 3159 3426 3456 3483
RI257 PROVIDENCE, RI 1869 1869 1869 1869 2034 2088 2178 2283 2481 2100 2223 2349 2535 2760 2199 2328 2565 2046 2085 2343 2838 3192 3219 3243
SC258 BEAUFORT/PARRIS ISLAND, SC 1659 1659 1659 1659 1749 1863 1995 2145 2265 1875 2058 2241 2277 2325 2028 2214 2283 1767 1860 2232 2334 2403 2424 2439
SC259 CHARLESTON, SC 1752 1752 1752 1752 1830 1917 1935 1962 2046 1929 1947 1980 2073 2190 1941 1974 2088 1845 1914 1977 2226 2406 2427 2442
SC260 COLUMBIA/FORT JACKSON, SC 1491 1491 1491 1491 1530 1566 1635 1719 1836 1578 1671 1773 1863 1974 1653 1755 1878 1542 1563 1767 2010 2184 2202 2217
SC261 GREENVILLE, SC 1359 1359 1359 1359 1395 1587 1602 1611 1713 1599 1608 1620 1749 1908 1605 1614 1770 1422 1584 1617 1962 2211 2229 2244
SC262 MYRTLE BEACH, SC 1434 1434 1434 1434 1533 1626 1725 1842 1971 1638 1773 1914 1995 2094 1749 1893 2007 1548 1623 1908 2124 2277 2295 2310
SC263 SUMTER/SHAW AFB, SC 1005 1005 1005 1005 1056 1290 1353 1422 1539 1305 1383 1467 1569 1695 1368 1452 1587 1089 1287 1461 1734 1932 1947 1959
SD264 RAPID CITY/ELLSWORTH AFB, SD 1224 1224 1224 1224 1377 1563 1578 1605 1662 1575 1590 1620 1680 1755 1584 1614 1689 1404 1560 1617 1776 1893 1905 1917
SD265 SIOUX FALLS, SD 1068 1068 1068 1068 1173 1455 1479 1509 1572 1470 1494 1530 1590 1668 1485 1524 1602 1212 1452 1527 1692 1809 1824 1833
TN266 CHATTANOOGA, TN 1389 1389 1389 1389 1401 1797 1815 1836 1920 1812 1824 1851 1950 2076 1821 1845 1968 1452 1794 1848 2115 2310 2328 2343
TN267 KNOXVILLE, TN 1269 1269 1269 1269 1353 1566 1659 1767 1893 1578 1707 1836 1917 2019 1683 1815 1932 1383 1563 1830 2052 2208 2226 2238
TN268 MEMPHIS, TN 1740 1740 1740 1740 1824 1968 1983 1992 2013 1980 1989 2001 2031 2097 1986 1995 2037 1848 1965 1998 2112 2211 2229 2244
TN269 NASHVILLE, TN 1899 1899 1899 1899 1917 2079 2094 2103 2151 2091 2100 2115 2166 2238 2097 2109 2175 1941 2076 2112 2256 2364 2382 2397
TN353 JOHNSON CITY/KINGSPORT, TN 978 978 978 978 1023 1272 1323 1383 1527 1284 1350 1422 1569 1746 1335 1410 1593 1056 1269 1419 1806 2088 2103 2118
TX270 ABILENE/DYESS AFB, TX 1140 1140 1140 1140 1236 1512 1560 1614 1716 1527 1584 1650 1743 1860 1572 1638 1758 1275 1509 1647 1896 2076 2091 2103
TX272 AUSTIN, TX 1908 1908 1908 1908 2028 2562 2727 2904 3021 2583 2799 3018 3024 3030 2763 2985 3027 2094 2556 3009 3033 3036 3039 3054
TX273 BEAUMONT, TX 1419 1419 1419 1419 1524 1671 1707 1755 1881 1683 1728 1785 1920 2082 1719 1776 1941 1545 1668 1782 2136 2388 2409 2424
TX274 COLLEGE STATION, TX 1410 1410 1410 1410 1467 1569 1614 1671 1785 1581 1641 1707 1818 1950 1626 1695 1836 1485 1566 1704 1995 2202 2220 2235
TX275 CORPUS CHRISTI, TX 1491 1491 1491 1491 1632 1869 1920 1977 2109 1881 1944 2016 2145 2304 1932 2004 2166 1665 1866 2013 2358 2604 2625 2646
TX277 DALLAS, TX 1737 1737 1737 1737 1932 2574 2595 2604 2616 2592 2601 2613 2619 2625 2598 2607 2622 2010 2565 2610 2628 2631 2634 2637
TX278 LAUGHLIN AFB/DEL RIO, TX 1080 1080 1080 1080 1227 1518 1575 1638 1740 1533 1602 1677 1767 1872 1587 1665 1779 1266 1515 1674 1908 2073 2091 2103
TX279 EL PASO, TX 1251 1251 1251 1251 1320 1527 1545 1554 1566 1542 1551 1563 1569 1587 1548 1557 1572 1347 1524 1560 1590 1626 1638 1647
TX281 BROWNSVILLE, TX 1164 1164 1164 1164 1194 1569 1623 1683 1803 1584 1647 1719 1836 1977 1635 1710 1854 1242 1566 1716 2025 2244 2262 2277
TX282 HOUSTON, TX 1602 1602 1602 1602 1671 1947 1965 1974 2004 1962 1971 1983 2016 2061 1968 1977 2019 1707 1944 1980 2073 2139 2157 2172
TX283 LUBBOCK, TX 1020 1020 1020 1020 1083 1578 1599 1608 1638 1596 1605 1617 1659 1740 1602 1611 1668 1146 1575 1614 1764 1890 1902 1914
TX284 GOODFELLOW AFB, TX 1182 1182 1182 1182 1323 1596 1650 1713 1797 1611 1677 1755 1812 1887 1665 1743 1821 1359 1593 1752 1908 2022 2037 2049
TX285 SAN ANTONIO, TX 1653 1653 1653 1653 1761 1788 1803 1812 1830 1800 1809 1821 1842 1905 1806 1815 1851 1770 1785 1818 1920 2013 2028 2040
TX286 FORT HOOD, TX 1140 1140 1140 1140 1143 1314 1407 1515 1623 1326 1452 1581 1641 1713 1428 1560 1647 1167 1311 1575 1734 1845 1860 1869
TX288 WICHITA FLS/SHEPPARD AFB, TX 1038 1038 1038 1038 1062 1284 1398 1527 1701 1299 1452 1608 1737 1896 1425 1584 1761 1092 1281 1602 1947 2193 2211 2226
TX356 FORT WORTH, TX 1584 1584 1584 1584 1653 1821 1866 1923 1992 1833 1890 1956 2007 2067 1878 1947 2013 1677 1818 1953 2085 2178 2193 2208
TX415 WACO,TX 1221 1221 1221 1221 1266 1416 1530 1662 1770 1428 1587 1746 1779 1821 1560 1722 1782 1290 1413 1740 1830 1890 1905 1917
UT291 OGDEN/HILL AFB, UT 1509 1509 1509 1509 1599 1620 1674 1743 1827 1632 1704 1785 1845 1917 1689 1773 1854 1608 1617 1782 1938 2049 2064 2079
UT292 SALT LAKE CITY, UT 1452 1452 1452 1452 1554 1659 1794 1944 2076 1671 1857 2040 2094 2157 1824 2013 2100 1572 1656 2034 2175 2274 2292 2307
UT357 PROVO, UT 1494 1494 1494 1494 1608 1629 1680 1749 1845 1641 1710 1788 1866 1965 1695 1776 1881 1617 1626 1785 1992 2142 2160 2172
VA295 CHARLOTTESVILLE, VA 1740 1740 1740 1740 1848 2124 2274 2442 2586 2139 2343 2547 2601 2670 2310 2517 2610 1884 2121 2538 2688 2790 2814 2835
VA296 QUANTICO/WOODBRIDGE, VA 1758 1758 1758 1758 1806 1839 1968 2112 2256 1851 2028 2205 2277 2364 1998 2178 2286 1815 1836 2199 2391 2523 2544 2562
VA297 HAMPTON/NEWPORT NEWS, VA 1473 1473 1473 1473 1518 1566 1581 1590 1683 1578 1587 1599 1722 1884 1584 1593 1746 1530 1563 1596 1938 2193 2208 2223
VA298 NORFOLK/PORTSMOUTH, VA 1734 1734 1734 1734 1737 1887 1926 1977 2088 1899 1947 2007 2118 2256 1938 1998 2139 1758 1884 2004 2301 2517 2538 2556
VA301 RICHMOND/FORT LEE, VA 1407 1407 1407 1407 1413 1689 1707 1716 1770 1704 1713 1725 1797 1911 1710 1719 1812 1449 1686 1722 1947 2127 2145 2157
VA302 WARRENTON, VA 2148 2148 2148 2148 2286 2379 2409 2451 2583 2391 2427 2475 2625 2805 2418 2466 2649 2301 2376 2472 2865 3150 3177 3201
VA303 LEXINGTON, VA 1068 1068 1068 1068 1098 1446 1464 1473 1527 1461 1470 1482 1554 1662 1467 1476 1569 1143 1443 1479 1695 1866 1878 1890
VA362 ROANOKE, VA 1257 1257 1257 1257 1362 1536 1611 1698 1785 1548 1647 1755 1800 1857 1629 1737 1806 1386 1533 1749 1872 1956 1971 1983
VA368 DAHLGREN/FORT AP HILL, VA 1767 1767 1767 1767 1827 1932 1947 1956 2052 1944 1953 1965 2088 2238 1950 1959 2109 1845 1929 1962 2289 2529 2550 2565
VT305 BURLINGTON, VT 1893 1893 1893 1893 1998 2538 2652 2775 2970 2556 2703 2853 3015 3207 2679 2832 3042 2064 2532 2847 3273 3579 3612 3639
WA306 BREMERTON, WA 1812 1812 1812 1812 1890 2130 2205 2289 2391 2145 2241 2343 2412 2499 2223 2325 2421 1923 2127 2337 2523 2655 2676 2697
WA307 EVERETT, WA 2133 2133 2133 2133 2286 2427 2481 2547 2634 2439 2508 2586 2655 2739 2496 2574 2664 2310 2424 2583 2763 2895 2919 2940
WA308 PORT ANGELES, WA 1413 1413 1413 1413 1548 1776 1857 1950 2049 1791 1896 2007 2067 2142 1875 1989 2076 1581 1773 2001 2163 2277 2295 2310
WA309 SEATTLE, WA 2508 2508 2508 2508 2679 3213 3249 3288 3411 3231 3267 3312 3453 3621 3258 3306 3474 2745 3210 3309 3678 3945 3981 4011
WA310 SPOKANE, WA 1275 1275 1275 1275 1437 1656 1740 1836 1932 1668 1782 1899 1950 2013 1761 1881 1956 1467 1653 1893 2031 2127 2142 2157
WA311 TACOMA, WA 1908 1908 1908 1908 2022 2214 2265 2328 2424 2226 2292 2367 2445 2544 2280 2355 2460 2049 2211 2364 2574 2727 2751 2769
WA312 WHIDBEY ISLAND, WA 1422 1422 1422 1422 1476 1860 1932 2010 2130 1878 1965 2058 2160 2283 1947 2043 2175 1524 1857 2055 2325 2517 2538 2556
WA313 YAKIMA, WA 1224 1224 1224 1224 1377 1692 1797 1914 1998 1707 1848 1989 2004 2028 1821 1968 2007 1419 1689 1983 2031 2064 2079 2091
WI316 MADISON, WI 1587 1587 1587 1587 1728 1770 1980 2220 2388 1782 2079 2373 2397 2427 2031 2328 2400 1740 1767 2361 2433 2478 2499 2514
WI317 MILWAUKEE, WI 1917 1917 1917 1917 2079 2100 2115 2124 2220 2112 2121 2133 2265 2460 2118 2127 2295 2088 2097 2130 2526 2829 2853 2874
WI318 SPARTA/FORT MCCOY, WI 1179 1179 1179 1179 1212 1428 1455 1494 1533 1440 1473 1515 1542 1578 1464 1509 1545 1242 1425 1512 1584 1635 1647 1656
WI359 STEVENS POINT, WI 966 966 966 966 1074 1350 1368 1386 1443 1365 1374 1398 1461 1539 1371 1392 1470 1110 1347 1395 1563 1683 1695 1707
WV320 MORGANTOWN, WV 1251 1251 1251 1251 1398 1560 1590 1626 1710 1572 1605 1650 1734 1839 1596 1644 1749 1422 1557 1647 1872 2034 2049 2061
WV322 HUNTINGTON, WV 1281 1281 1281 1281 1338 1521 1578 1647 1737 1533 1608 1689 1758 1842 1593 1677 1767 1365 1518 1686 1869 1998 2013 2025
WV323 CHARLESTON, WV 984 984 984 984 1092 1176 1191 1200 1242 1188 1197 1209 1263 1362 1194 1203 1275 1107 1173 1206 1389 1539 1551 1560
WV454 EASTERN PANHANDLE, WV 1446 1446 1446 1446 1569 1623 1725 1845 1977 1635 1773 1920 1998 2100 1749 1896 2013 1581 1620 1914 2130 2283 2301 2316
WY324 CHEYENNE, WY 1038 1038 1038 1038 1191 1515 1605 1704 1785 1530 1647 1767 1794 1830 1626 1749 1797 1233 1512 1764 1836 1884 1899 1911
ZZ510 COUNTY COST GROUP 510 1011 1011 1011 1011 1083 1281 1326 1383 1467 1293 1350 1419 1488 1575 1338 1407 1500 1110 1278 1416 1602 1734 1746 1758
ZZ520 COUNTY COST GROUP 520 1020 1020 1020 1020 1089 1287 1335 1392 1476 1302 1359 1428 1497 1584 1347 1416 1509 1119 1284 1425 1611 1743 1758 1767
ZZ530 COUNTY COST GROUP 530 1026 1026 1026 1026 1098 1296 1344 1401 1485 1311 1368 1437 1506 1596 1356 1425 1518 1128 1293 1434 1620 1755 1767 1779
ZZ540 COUNTY COST GROUP 540 1035 1035 1035 1035 1107 1305 1353 1410 1497 1320 1377 1446 1518 1605 1365 1434 1527 1134 1302 1443 1629 1764 1779 1788
ZZ550 COUNTY COST GROUP 550 1044 1044 1044 1044 1116 1314 1362 1419 1506 1329 1386 1455 1527 1614 1374 1443 1536 1143 1311 1452 1641 1776 1788 1800
ZZ560 COUNTY COST GROUP 560 1050 1050 1050 1050 1122 1323 1371 1428 1515 1335 1395 1464 1536 1623 1383 1452 1545 1152 1320 1461 1650 1785 1797 1809
ZZ570 COUNTY COST GROUP 570 1056 1056 1056 1056 1131 1332 1380 1437 1521 1344 1404 1470 1542 1632 1392 1461 1554 1158 1329 1467 1659 1794 1806 1818
ZZ580 COUNTY COST GROUP 580 1065 1065 1065 1065 1137 1338 1386 1443 1530 1353 1413 1479 1551 1641 1398 1470 1563 1167 1335 1476 1668 1803 1818 1827
ZZ590 COUNTY COST GROUP 590 1071 1071 1071 1071 1143 1347 1395 1452 1539 1359 1419 1488 1560 1647 1407 1476 1569 1173 1344 1485 1674 1812 1824 1836
ZZ600 COUNTY COST GROUP 600 1077 1077 1077 1077 1149 1353 1401 1458 1545 1365 1425 1494 1566 1656 1413 1482 1578 1179 1350 1491 1683 1818 1833 1842
ZZ610 COUNTY COST GROUP 610 1083 1083 1083 1083 1158 1362 1410 1467 1554 1374 1434 1503 1575 1665 1422 1491 1587 1188 1359 1500 1692 1830 1842 1854
ZZ620 COUNTY COST GROUP 620 1092 1092 1092 1092 1167 1371 1419 1476 1563 1383 1443 1512 1584 1674 1431 1500 1596 1194 1368 1509 1701 1839 1854 1863
ZZ630 COUNTY COST GROUP 630 1101 1101 1101 1101 1176 1380 1428 1485 1572 1392 1452 1521 1596 1686 1440 1512 1605 1203 1377 1518 1713 1851 1866 1875
ZZ640 COUNTY COST GROUP 640 1107 1107 1107 1107 1185 1389 1437 1494 1581 1401 1461 1530 1605 1695 1449 1521 1614 1212 1386 1527 1722 1860 1875 1887
ZZ650 COUNTY COST GROUP 650 1116 1116 1116 1116 1194 1398 1446 1506 1593 1413 1473 1542 1614 1707 1461 1530 1626 1221 1395 1539 1734 1872 1887 1899
ZZ660 COUNTY COST GROUP 660 1125 1125 1125 1125 1203 1407 1458 1515 1602 1422 1482 1551 1626 1716 1470 1539 1638 1230 1404 1548 1743 1884 1899 1911
ZZ670 COUNTY COST GROUP 670 1134 1134 1134 1134 1212 1419 1467 1524 1614 1431 1491 1560 1635 1728 1479 1551 1647 1242 1416 1557 1755 1896 1911 1923
ZZ680 COUNTY COST GROUP 680 1143 1143 1143 1143 1221 1428 1476 1536 1623 1440 1503 1572 1647 1737 1488 1560 1656 1251 1425 1569 1764 1908 1920 1932
ZZ690 COUNTY COST GROUP 690 1149 1149 1149 1149 1227 1437 1485 1542 1632 1449 1509 1578 1653 1746 1497 1569 1665 1257 1434 1575 1773 1917 1932 1941
ZZ700 COUNTY COST GROUP 700 1155 1155 1155 1155 1233 1443 1491 1548 1638 1455 1515 1587 1662 1752 1503 1575 1671 1263 1440 1584 1782 1923 1938 1950
ZZ710 COUNTY COST GROUP 710 1161 1161 1161 1161 1242 1449 1500 1557 1647 1464 1524 1593 1668 1761 1512 1584 1680 1272 1446 1590 1791 1932 1947 1959
ZZ720 COUNTY COST GROUP 720 1173 1173 1173 1173 1251 1461 1509 1569 1659 1473 1536 1605 1680 1773 1524 1593 1692 1281 1458 1602 1803 1944 1959 1971
ZZ730 COUNTY COST GROUP 730 1182 1182 1182 1182 1263 1473 1521 1581 1671 1485 1548 1617 1692 1788 1533 1605 1704 1293 1470 1614 1815 1959 1974 1986
ZZ740 COUNTY COST GROUP 740 1197 1197 1197 1197 1278 1488 1539 1596 1686 1503 1563 1635 1710 1803 1551 1623 1722 1308 1485 1629 1833 1977 1992 2004
ZZ750 COUNTY COST GROUP 750 1221 1221 1221 1221 1302 1515 1566 1626 1716 1530 1590 1662 1740 1833 1578 1650 1752 1332 1512 1659 1863 2010 2025 2037
ZZ760 COUNTY COST GROUP 760 1242 1242 1242 1242 1326 1542 1590 1650 1743 1554 1617 1689 1764 1863 1605 1677 1779 1356 1539 1683 1890 2040 2055 2067
ZZ770 COUNTY COST GROUP 770 1266 1266 1266 1266 1353 1569 1617 1677 1770 1581 1644 1716 1794 1890 1632 1704 1806 1383 1566 1713 1920 2070 2088 2100
ZZ780 COUNTY COST GROUP 780 1296 1296 1296 1296 1383 1602 1653 1713 1806 1614 1677 1749 1830 1929 1665 1740 1842 1413 1599 1746 1959 2109 2127 2139
ZZ790 COUNTY COST GROUP 790 1320 1320 1320 1320 1407 1629 1680 1740 1833 1641 1704 1776 1857 1956 1692 1767 1869 1440 1626 1773 1986 2139 2157 2172
ZZ800 COUNTY COST GROUP 800 1356 1356 1356 1356 1446 1668 1719 1779 1875 1680 1746 1818 1899 2001 1731 1806 1911 1476 1665 1815 2031 2187 2202 2217
ZZ810 COUNTY COST GROUP 810 1389 1389 1389 1389 1482 1704 1758 1818 1914 1719 1782 1857 1941 2043 1770 1845 1953 1512 1701 1854 2073 2232 2247 2262
ZZ820 COUNTY COST GROUP 820 1434 1434 1434 1434 1530 1755 1809 1872 1968 1770 1836 1908 1995 2097 1821 1899 2007 1560 1752 1905 2130 2289 2310 2325
ZZ830 COUNTY COST GROUP 830 1488 1488 1488 1488 1584 1815 1869 1932 2031 1830 1896 1971 2058 2163 1881 1959 2070 1617 1812 1968 2196 2361 2379 2394
ZZ840 COUNTY COST GROUP 840 1536 1536 1536 1536 1635 1869 1923 1986 2088 1884 1950 2028 2115 2223 1938 2016 2130 1668 1866 2022 2256 2424 2442 2460
ZZ850 COUNTY COST GROUP 850 1599 1599 1599 1599 1704 1941 1995 2061 2163 1956 2022 2100 2190 2301 2010 2088 2205 1734 1938 2097 2334 2508 2526 2544
ZZ860 COUNTY COST GROUP 860 1662 1662 1662 1662 1770 2013 2067 2133 2238 2025 2097 2172 2265 2379 2082 2160 2280 1803 2010 2169 2415 2589 2610 2628
ZZ870 COUNTY COST GROUP 870 1749 1749 1749 1749 1863 2112 2169 2235 2343 2124 2196 2274 2370 2487 2181 2262 2385 1896 2109 2271 2523 2706 2727 2748
"""

withDepend = withDependstr.strip().split('\n')

In [414]:
withDependComb = {}

for line in withDepend:
    if line.startswith("ZZ"):
        continue
    left = line.split()
    mha = left[0]
    withDependComb[mha] = {}
    i = 0
    for cell in left[1:]:
        if cell.isnumeric():
            try:
                withDependComb[mha][ranks[i]] = float(cell)
            except Exception as e:
                print(e)
                print(cell)
                print(i)
                print(line)
            i += 1

In [415]:
withoutDependstr = """
AK400 KETCHIKAN, AK 1380 1380 1380 1380 1662 1674 1743 1908 2022 1677 1905 2034 2235 2352 1830 1998 2205 1671 1764 2070 2331
AK401 SITKA, AK 1455 1455 1455 1455 1758 1767 1773 2013 2133 1770 2010 2148 2340 2370 1929 2109 2331 1764 1854 2187 2361
AK402 JUNEAU, AK 1401 1401 1401 1401 1878 1887 1941 2022 2178 1890 1983 2199 2514 2613 1959 2136 2490 1884 1944 2262 2592
AK403 KODIAK ISLAND, AK 1248 1248 1248 1248 1536 1548 1635 1752 1881 1560 1731 1884 2073 2214 1659 1824 2037 1545 1638 1899 2187
AK404 ANCHORAGE, AK 1281 1281 1281 1281 1566 1575 1689 1833 1956 1581 1776 1959 2118 2295 1719 1896 2076 1572 1692 1962 2262
AK405 FAIRBANKS, AK 1098 1098 1098 1098 1500 1509 1572 1653 1773 1512 1605 1776 2013 2118 1590 1692 1986 1506 1575 1797 2097
AL001 ANNISTON/FORT MCCLELLAN, AL 708 708 708 708 819 906 933 1026 1092 927 1023 1098 1206 1212 975 1077 1203 885 963 1122 1209
AL002 FORT RUCKER, AL 906 906 906 906 969 1014 1062 1149 1275 1056 1128 1278 1305 1389 1110 1209 1302 1011 1098 1281 1386
AL003 HUNTSVILLE, AL 1074 1074 1074 1074 1206 1284 1359 1494 1572 1341 1491 1581 1710 1716 1431 1554 1707 1281 1410 1611 1713
AL004 MOBILE, AL 1134 1134 1134 1134 1221 1284 1395 1512 1518 1359 1509 1521 1551 1668 1500 1515 1527 1275 1464 1524 1644
AL005 MONTGOMERY, AL 1011 1011 1011 1011 1089 1140 1194 1257 1263 1185 1254 1266 1278 1338 1245 1260 1272 1137 1230 1269 1323
AL006 AUBURN, AL 912 912 912 912 1008 1068 1125 1221 1338 1113 1209 1341 1365 1497 1179 1275 1362 1065 1164 1344 1470
AL007 BIRMINGHAM, AL 1095 1095 1095 1095 1236 1332 1464 1617 1632 1419 1614 1635 1668 1674 1596 1629 1665 1320 1554 1641 1671
AR010 LITTLE ROCK, AR 1038 1038 1038 1038 1194 1299 1386 1485 1491 1362 1482 1494 1512 1530 1470 1488 1506 1293 1443 1497 1521
AR012 FORT CHAFFEE/FORT SMITH, AR 657 657 657 657 753 861 933 1011 1077 873 966 1080 1170 1263 951 1044 1143 810 936 1083 1245
AR411 FAYETTEVILLE, AR 768 768 768 768 867 996 1053 1164 1221 1008 1161 1227 1341 1422 1116 1209 1323 927 1086 1251 1404
AZ013 PHOENIX, AZ 1320 1320 1320 1320 1431 1503 1608 1758 1818 1578 1755 1824 1935 1983 1710 1806 1923 1497 1677 1848 1968
AZ014 FORT HUACHUCA, AZ 936 936 936 936 963 1068 1089 1200 1281 1080 1197 1290 1428 1455 1137 1263 1419 984 1116 1320 1446
AZ015 DAVIS-MONTHAN AFB, AZ 1002 1002 1002 1002 1104 1284 1314 1503 1581 1299 1500 1587 1713 1719 1446 1563 1710 1167 1401 1617 1716
AZ016 YUMA, AZ 822 822 822 822 909 1053 1083 1179 1263 1068 1176 1272 1413 1452 1116 1245 1401 963 1092 1302 1440
CA018 OAKLAND, CA 2688 2688 2688 2688 2952 3102 3234 3426 3507 3189 3423 3516 3657 3699 3366 3489 3645 3090 3321 3546 3687
CA019 SAN FRANCISCO, CA 3402 3402 3402 3402 3822 4149 4560 4995 5013 4383 4992 5016 5058 5067 4971 5010 5055 4098 4827 5025 5061
CA021 CHINA LAKE, CA 762 762 762 762 864 957 1065 1179 1293 972 1113 1296 1323 1452 1086 1230 1320 924 1068 1299 1425
CA022 FRESNO, CA 1254 1254 1254 1254 1344 1401 1500 1632 1677 1470 1629 1683 1776 1854 1593 1668 1758 1395 1563 1701 1836
CA023 LEMOORE NAS, CA 1125 1125 1125 1125 1320 1431 1476 1569 1629 1470 1566 1635 1737 1743 1518 1614 1734 1428 1506 1659 1740
CA024 CAMP PENDLETON, CA 2103 2103 2103 2103 2265 2391 2586 2877 2994 2502 2874 3006 3189 3195 2793 2967 3186 2352 2721 3045 3192
CA025 VENTURA, CA 2145 2145 2145 2145 2400 2541 2616 2757 2847 2598 2754 2856 3003 3030 2688 2826 2997 2538 2667 2886 3021
CA026 VANDENBERG AFB, CA 1671 1671 1671 1671 1836 1944 2106 2295 2325 2046 2292 2328 2382 2391 2265 2319 2379 1929 2211 2340 2385
CA027 MARIN/SONOMA, CA 2685 2685 2685 2685 2919 3051 3144 3333 3483 3117 3330 3498 3726 3732 3231 3450 3723 3045 3204 3549 3729
CA028 BARSTOW/FORT IRWIN, CA 1068 1068 1068 1068 1188 1260 1311 1401 1452 1302 1398 1458 1557 1632 1359 1440 1539 1257 1344 1476 1614
CA031 SAN BERNARDINO, CA 1839 1839 1839 1839 1953 2085 2238 2502 2607 2163 2499 2616 2778 2787 2430 2583 2775 2022 2364 2652 2781
CA032 TWENTY NINE PALMS MCB, CA 948 948 948 948 1011 1059 1134 1251 1308 1113 1248 1317 1431 1512 1206 1296 1410 1053 1182 1338 1494
CA033 BEALE AFB, CA 1689 1689 1689 1689 1830 1917 2016 2160 2214 1986 2157 2220 2313 2322 2112 2202 2310 1911 2082 2241 2316
CA034 SACRAMENTO, CA 1689 1689 1689 1689 1848 1953 2121 2316 2346 2058 2313 2349 2412 2433 2286 2340 2406 1938 2229 2364 2424
CA035 STOCKTON, CA 1320 1320 1320 1320 1539 1659 1692 1746 1776 1689 1743 1782 1848 1896 1713 1770 1836 1656 1707 1794 1884
CA036 VALLEJO/TRAVIS AFB, CA 2142 2142 2142 2142 2265 2355 2541 2736 2742 2472 2733 2745 2754 2760 2727 2739 2751 2337 2664 2748 2757
CA037 LOS ANGELES, CA 2256 2256 2256 2256 2517 2694 3024 3384 3414 2883 3381 3420 3477 3489 3354 3408 3474 2652 3237 3432 3480
CA038 SAN DIEGO, CA 2019 2019 2019 2019 2268 2460 2703 3003 3069 2604 3000 3078 3195 3234 2949 3054 3186 2430 2865 3102 3222
CA039 MONTEREY, CA 1818 1818 1818 1818 2157 2340 2505 2706 2895 2400 2616 2898 3150 3399 2547 2802 3087 2337 2508 2901 3357
CA041 RIVERSIDE, CA 1788 1788 1788 1788 1911 1989 2094 2235 2274 2064 2232 2277 2352 2370 2199 2265 2346 1983 2166 2295 2361
CA042 HUMBOLDT COUNTY, CA 1101 1101 1101 1101 1200 1338 1431 1539 1671 1353 1536 1674 1812 1938 1473 1611 1782 1257 1440 1677 1914
CA044 SANTA CLARA COUNTY, CA 3009 3009 3009 3009 3384 3609 3864 4239 4416 3759 4236 4434 4731 4830 4119 4380 4704 3579 4029 4494 4809
CA392 SAN LUIS OBISPO, CA 1554 1554 1554 1554 1773 2172 2202 2361 2565 2184 2358 2586 2904 2940 2226 2520 2892 1920 2205 2655 2928
CA393 BRIDGEPORT, CA 1005 1005 1005 1005 1155 1473 1491 1524 1692 1488 1521 1710 1965 1974 1497 1656 1962 1269 1494 1767 1968
CA420 ELCENTRO, CA 1068 1068 1068 1068 1188 1260 1320 1401 1416 1308 1398 1419 1464 1497 1380 1413 1455 1257 1362 1428 1485
CA457 EDWARDS AFB/PALMDALE, CA 1515 1515 1515 1515 1698 1803 1857 1938 1959 1848 1935 1962 2040 2175 1911 1956 2004 1800 1896 1974 2148
CO045 DENVER, CO 1500 1500 1500 1500 1638 1854 1956 2235 2322 1872 2232 2331 2472 2478 2169 2304 2469 1719 2097 2361 2475
CO046 COLORADO SPRINGS, CO 1431 1431 1431 1431 1608 1713 1833 1965 1974 1794 1962 1977 1998 2016 1947 1971 1992 1704 1908 1980 2007
CO047 FORT COLLINS, CO 1413 1413 1413 1413 1491 1542 1626 1740 1779 1605 1737 1785 1872 1953 1704 1770 1851 1539 1680 1800 1935
CO422 BOULDER, CO 1491 1491 1491 1491 1692 1950 1998 2262 2388 1962 2259 2400 2607 2652 2172 2361 2595 1803 2115 2445 2640
CT049 NEW LONDON, CT 1248 1248 1248 1248 1335 1518 1533 1647 1788 1530 1644 1803 2022 2031 1548 1758 2019 1386 1536 1851 2025
CT050 HARTFORD, CT 1416 1416 1416 1416 1692 1851 1887 1977 2043 1884 1974 2052 2175 2229 1923 2031 2160 1848 1914 2076 2217
CT051 NEW HAVEN/FAIRFIELD, CT 2184 2184 2184 2184 2559 2754 2769 2805 2874 2766 2802 2880 2994 3000 2775 2859 2991 2751 2772 2907 2997
DC053 WASHINGTON, DC METRO AREA 2061 2061 2061 2061 2340 2493 2523 2580 2619 2520 2577 2625 2736 2907 2544 2610 2691 2490 2538 2640 2877
DE054 DOVER AFB/REHOBOTH, DE 1248 1248 1248 1248 1437 1542 1605 1677 1683 1593 1674 1686 1701 1797 1668 1680 1692 1539 1650 1689 1779
FL056 EGLIN AFB, FL 1371 1371 1371 1371 1533 1626 1653 1692 1716 1650 1689 1719 1794 1902 1665 1710 1764 1623 1662 1731 1881
FL057 GAINESVILLE, FL 1230 1230 1230 1230 1368 1449 1464 1482 1515 1461 1479 1521 1614 1734 1470 1509 1584 1446 1467 1536 1713
FL058 JACKSONVILLE, FL 1320 1320 1320 1320 1470 1557 1617 1707 1737 1605 1704 1743 1824 1908 1674 1731 1800 1554 1656 1758 1890
FL059 PATRICK AFB, FL 1479 1479 1479 1479 1596 1671 1758 1851 1857 1734 1848 1860 1875 1929 1839 1854 1866 1665 1812 1863 1914
FL061 MIAMI/FORT LAUDERDALE, FL 1932 1932 1932 1932 2181 2319 2343 2421 2511 2340 2418 2523 2682 2742 2352 2493 2664 2316 2349 2553 2727
FL062 ORLANDO, FL 1455 1455 1455 1455 1701 1836 1881 1956 1983 1875 1953 1989 2043 2049 1926 1977 2040 1833 1914 2001 2046
FL063 PANAMA CITY, FL 1515 1515 1515 1515 1587 1635 1716 1827 1863 1695 1824 1869 1962 2073 1791 1857 1935 1632 1767 1884 2049
FL064 PENSACOLA, FL 1191 1191 1191 1191 1335 1419 1461 1551 1608 1455 1548 1614 1719 1758 1503 1596 1707 1416 1491 1635 1746
FL065 TALLAHASSEE, FL 1236 1236 1236 1236 1380 1467 1485 1497 1503 1482 1494 1506 1530 1671 1491 1500 1527 1464 1488 1509 1668
FL066 TAMPA, FL 1542 1542 1542 1542 1731 1842 1962 2103 2115 1923 2100 2118 2148 2157 2082 2112 2145 1833 2043 2124 2151
FL067 WEST PALM BEACH, FL 1671 1671 1671 1671 1887 2013 2121 2238 2244 2085 2235 2247 2271 2337 2226 2241 2253 2004 2190 2250 2322
FL068 OCALA, FL 1221 1221 1221 1221 1353 1431 1482 1542 1548 1473 1539 1551 1560 1566 1530 1545 1557 1428 1515 1554 1563
FL069 FLORIDA KEYS, FL 2121 2121 2121 2121 2391 2547 2682 2925 3087 2637 2922 3102 3393 3576 2817 3051 3345 2535 2772 3156 3543
FL070 VOLUSIA COUNTY, FL 1275 1275 1275 1275 1431 1527 1641 1764 1770 1605 1761 1773 1782 1800 1752 1767 1779 1518 1716 1776 1788
FL423 FORT PIERCE, FL 1506 1506 1506 1506 1509 1527 1689 1860 1866 1629 1857 1869 1878 1884 1848 1863 1875 1512 1794 1872 1881
FL424 FT MYERS BCH, FL 1398 1398 1398 1398 1491 1551 1623 1707 1713 1605 1704 1716 1749 1824 1689 1710 1728 1548 1668 1719 1809
GA071 ATLANTA, GA 1638 1638 1638 1638 1866 1998 2130 2289 2319 2085 2286 2322 2379 2385 2259 2313 2376 1986 2214 2334 2382
GA072 ALBANY, GA 786 786 786 786 906 978 1011 1047 1101 1008 1044 1104 1143 1275 1035 1050 1140 975 1029 1107 1272
GA073 FORT GORDON, GA 1224 1224 1224 1224 1341 1410 1434 1476 1509 1431 1473 1512 1578 1605 1443 1503 1572 1407 1440 1527 1596
GA074 KINGS BAY/BRUNSWICK, GA 1164 1164 1164 1164 1224 1269 1323 1395 1404 1314 1392 1407 1434 1455 1377 1401 1428 1266 1362 1410 1443
GA075 FORT BENNING, GA 972 972 972 972 1080 1167 1182 1275 1380 1179 1272 1389 1554 1560 1200 1356 1551 1143 1194 1425 1557
GA076 ROBINS AFB, GA 996 996 996 996 1092 1149 1230 1314 1320 1209 1311 1323 1332 1389 1302 1317 1329 1146 1281 1326 1374
GA077 SAVANNAH, GA 1209 1209 1209 1209 1311 1377 1470 1602 1653 1443 1599 1659 1746 1755 1557 1641 1743 1371 1530 1680 1749
GA079 DAHLONEGA, GA 1182 1182 1182 1182 1359 1554 1569 1605 1782 1566 1602 1800 2070 2076 1575 1743 2067 1458 1572 1860 2073
GA080 FORT STEWART, GA 1122 1122 1122 1122 1260 1344 1431 1521 1527 1407 1518 1530 1539 1605 1512 1524 1536 1341 1485 1533 1590
GA081 MOODY AFB, GA 954 954 954 954 996 1029 1059 1107 1131 1056 1104 1134 1203 1296 1080 1125 1179 1026 1074 1143 1278
HI407 MAUI COUNTY, HI 2034 2034 2034 2034 2145 2544 2634 2967 3129 2556 2964 3144 3420 3537 2859 3093 3387 2196 2748 3198 3516
HI408 HONOLULU COUNTY, HI 2118 2118 2118 2118 2412 2667 2829 3000 3006 2769 2997 3009 3048 3234 2991 3003 3015 2652 2937 3012 3201
HI409 HAWAII COUNTY, HI 1605 1605 1605 1605 1875 2058 2142 2268 2325 2118 2265 2331 2427 2442 2223 2313 2424 2052 2196 2352 2433
HI414 KAUAI COUNTY,HI 1872 1872 1872 1872 2058 2247 2451 2799 2871 2328 2796 2877 2994 3000 2742 2853 2991 2121 2640 2904 2997
IA082 DES MOINES, IA 951 951 951 951 1065 1269 1320 1452 1542 1284 1449 1551 1707 1770 1386 1524 1689 1140 1347 1584 1755
ID084 BOISE, ID 1239 1239 1239 1239 1344 1410 1431 1476 1533 1428 1473 1539 1638 1674 1437 1521 1629 1407 1434 1560 1662
ID086 MOUNTAIN HOME AFB, ID 1071 1071 1071 1071 1125 1368 1386 1569 1665 1383 1566 1674 1827 1848 1500 1644 1821 1158 1443 1707 1839
IL088 CHAMPAIGN/URBANA, IL 867 867 867 867 930 1014 1032 1146 1221 1026 1143 1230 1353 1377 1089 1206 1347 969 1071 1257 1368
IL089 ROCK ISLAND, IL 1149 1149 1149 1149 1272 1419 1533 1674 1701 1497 1671 1704 1785 1902 1647 1695 1755 1410 1608 1716 1881
IL090 PEORIA, IL 843 843 843 843 972 1284 1302 1320 1449 1296 1311 1467 1713 1737 1308 1416 1707 1068 1305 1521 1728
IL092 GREAT LAKES NAVTRACEN, IL 1302 1302 1302 1302 1401 1611 1683 1761 1884 1626 1716 1893 2169 2265 1698 1842 2145 1458 1686 1950 2244
IL093 SCOTT AFB, IL 1035 1035 1035 1035 1071 1101 1164 1263 1305 1152 1260 1311 1410 1503 1221 1296 1386 1098 1206 1329 1482
IL325 CHICAGO, IL 1683 1683 1683 1683 1740 1791 1926 2106 2166 1881 2103 2172 2277 2301 2058 2154 2271 1779 2016 2196 2292
IL335 SPRINGFIELD/DECATUR,IL 741 741 741 741 807 1131 1149 1161 1251 1146 1158 1269 1509 1518 1155 1218 1506 852 1152 1320 1512
IN094 INDIANAPOLIS, IN 1083 1083 1083 1083 1266 1377 1443 1518 1524 1431 1515 1527 1536 1548 1506 1521 1533 1374 1488 1530 1539
IN097 FORT WAYNE, IN 993 993 993 993 1149 1158 1332 1539 1545 1254 1536 1548 1557 1605 1530 1542 1554 1155 1461 1551 1590
IN338 TERRE HAUTE, IN 714 714 714 714 846 927 957 987 993 954 984 996 1005 1017 975 990 1002 924 969 999 1008
IN399 BLOOMINGTON, IN 918 918 918 918 1020 1086 1173 1287 1380 1149 1284 1383 1440 1626 1257 1308 1437 1083 1230 1386 1623
KS100 FORT RILEY, KS 876 876 876 876 936 981 1050 1158 1209 1032 1155 1215 1317 1377 1113 1197 1299 978 1095 1236 1362
KS101 WICHITA/MCCONNELL AFB, KS 801 801 801 801 912 1023 1038 1083 1188 1035 1080 1197 1362 1368 1044 1164 1359 975 1041 1233 1365
KS102 FORT LEAVENWORTH, KS 1122 1122 1122 1122 1281 1398 1497 1608 1620 1467 1605 1623 1650 1668 1590 1617 1644 1392 1560 1626 1659
KS105 TOPEKA, KS 828 828 828 828 876 1077 1095 1224 1302 1092 1221 1311 1437 1446 1167 1287 1434 906 1125 1338 1440
KY106 FORT CAMPBELL, KY 1098 1098 1098 1098 1245 1332 1362 1410 1437 1359 1407 1440 1518 1623 1380 1431 1491 1329 1374 1452 1602
KY107 LEXINGTON, KY 969 969 969 969 1083 1152 1227 1326 1350 1209 1323 1353 1404 1419 1299 1347 1401 1149 1278 1365 1410
KY109 LOUISVILLE, KY 1080 1080 1080 1080 1227 1320 1446 1590 1605 1404 1587 1608 1665 1749 1566 1602 1644 1311 1527 1617 1731
KY110 FORT KNOX, KY 804 804 804 804 900 1029 1095 1167 1248 1041 1125 1251 1392 1479 1110 1197 1368 957 1098 1254 1458
KY339 FRANKFORT, KY 810 810 810 810 837 861 906 999 1113 888 948 1116 1152 1269 927 1041 1149 858 909 1119 1266
KY430 PADUCAH, KY 732 732 732 732 864 942 963 1005 1065 960 1002 1071 1185 1242 969 1053 1167 939 966 1095 1227
LA113 ALEXANDRIA, LA 714 714 714 714 804 969 984 1002 1110 981 993 1113 1293 1371 990 1071 1287 858 987 1149 1368
LA114 BATON ROUGE, LA 1077 1077 1077 1077 1230 1323 1434 1587 1647 1398 1584 1653 1767 1842 1539 1635 1749 1314 1506 1674 1824
LA115 FORT POLK, LA 699 699 699 699 741 975 999 1056 1146 987 1053 1155 1305 1338 1005 1125 1296 771 1002 1188 1326
LA116 NEW ORLEANS, LA 1104 1104 1104 1104 1236 1359 1464 1686 1731 1395 1683 1734 1830 1908 1647 1719 1809 1260 1584 1752 1893
LA117 SHREVEPORT/BARKSDALE AFB, LA 1074 1074 1074 1074 1164 1356 1371 1509 1620 1368 1506 1632 1806 1812 1428 1596 1803 1218 1398 1671 1809
LA118 LAFAYETTE, LA 1032 1032 1032 1032 1170 1254 1326 1455 1533 1308 1452 1542 1668 1677 1395 1518 1665 1251 1374 1569 1671
LA326 ST MARY AND TERREBONNE, LA 903 903 903 903 1020 1104 1206 1329 1347 1176 1326 1350 1404 1467 1305 1344 1386 1098 1272 1359 1452
LA370 LAKE CHARLES, LA 1125 1125 1125 1125 1224 1482 1500 1596 1740 1497 1593 1755 1977 1983 1506 1710 1974 1281 1503 1803 1980
LA371 MONROE, LA 732 732 732 732 828 969 999 1032 1122 981 1020 1134 1299 1338 1005 1101 1287 885 1002 1167 1326
MA119 NANTUCKET, MA 2367 2367 2367 2367 2625 2961 3165 3459 3585 3087 3456 3600 3846 4050 3369 3558 3795 2940 3300 3642 4017
MA120 BOSTON, MA 2433 2433 2433 2433 2634 2757 2916 3165 3297 2859 3162 3309 3534 3615 3075 3270 3510 2742 3021 3354 3597
MA122 WORCESTER, MA 1716 1716 1716 1716 1914 2028 2049 2169 2343 2046 2166 2361 2637 2670 2055 2307 2628 2025 2052 2421 2658
MA123 FITCHBURG, MA 1665 1665 1665 1665 1842 2022 2100 2337 2472 2046 2334 2487 2700 2721 2244 2442 2694 1938 2196 2532 2712
MA124 CAPE COD-PLYMOUTH, MA 2109 2109 2109 2109 2253 2340 2373 2460 2538 2370 2457 2544 2688 2766 2400 2520 2667 2337 2394 2574 2748
MA125 ESSEX CO, MA 1890 1890 1890 1890 2061 2169 2337 2571 2664 2274 2568 2676 2832 2862 2502 2646 2823 2154 2445 2709 2850
MA126 HAMPDEN COUNTY, MA 1173 1173 1173 1173 1362 1527 1590 1731 1845 1539 1728 1857 2052 2139 1650 1818 2031 1470 1623 1896 2121
MA151 MARTHA'S VINEYARD, MA 1836 1836 1836 1836 2061 2313 2463 2820 2916 2346 2817 2928 3108 3210 2748 2895 3081 2145 2649 2961 3189
MA377 HANSCOM AFB, MA 2052 2052 2052 2052 2343 2505 2601 2784 2916 2571 2781 2928 3153 3243 2694 2886 3129 2499 2664 2973 3225
MD127 ABERDEEN PROVING GROUNDS, MD 1437 1437 1437 1437 1539 1608 1716 1863 1908 1683 1860 1914 2013 2100 1821 1899 1989 1599 1788 1932 2079
MD128 ANNAPOLIS, MD 1797 1797 1797 1797 1989 2100 2193 2337 2409 2166 2334 2418 2535 2541 2283 2394 2532 2097 2253 2442 2538
MD129 BALTIMORE, MD 1638 1638 1638 1638 1704 1764 1917 2127 2199 1863 2124 2208 2334 2385 2070 2184 2322 1749 2019 2232 2370
MD130 FORT DETRICK, MD 1491 1491 1491 1491 1584 1644 1737 1884 1959 1710 1881 1968 2094 2103 1824 1944 2091 1638 1797 1995 2097
MD133 FORT G. G. MEADE, MD 1803 1803 1803 1803 1932 2010 2100 2256 2346 2076 2253 2355 2508 2544 2190 2328 2496 2004 2160 2388 2532
MD134 INDIAN HEAD NAVORDSTA, MD 1788 1788 1788 1788 1908 1998 2169 2367 2391 2103 2364 2394 2448 2466 2337 2385 2445 1980 2280 2406 2457
MD135 PATUXENT RIVER, MD 1476 1476 1476 1476 1545 1596 1695 1857 1947 1665 1854 1959 2103 2109 1791 1929 2100 1590 1758 1989 2106
MD432 OCEAN CITY, MD 1086 1086 1086 1086 1236 1326 1344 1392 1464 1341 1389 1470 1593 1626 1350 1449 1584 1323 1347 1497 1617
MD458 OXFORD,MD 1266 1266 1266 1266 1428 1677 1695 1974 2070 1689 1971 2082 2235 2241 1902 2049 2232 1464 1830 2115 2238
ME136 BRUNSWICK, ME 1026 1026 1026 1026 1221 1293 1617 1971 2244 1311 1764 2247 2274 2367 1689 2133 2271 1236 1620 2250 2364
ME139 PORTLAND, ME 1737 1737 1737 1737 1941 2052 2067 2160 2319 2064 2157 2337 2589 2628 2073 2286 2580 2049 2070 2391 2616
ME141 COASTAL MAINE, ME 1131 1131 1131 1131 1290 1410 1479 1758 1800 1425 1755 1806 1884 1905 1719 1791 1878 1308 1635 1821 1896
ME390 BANGOR, ME 867 867 867 867 1008 1236 1251 1410 1497 1248 1407 1506 1647 1659 1344 1479 1644 1041 1284 1536 1650
MI142 DETROIT, MI 1302 1302 1302 1302 1473 1698 1749 2028 2115 1713 2025 2124 2265 2271 1965 2097 2262 1506 1890 2157 2268
MI143 MARQUETTE, MI 894 894 894 894 1041 1233 1266 1305 1431 1248 1299 1443 1653 1698 1275 1401 1641 1164 1269 1488 1686
MI145 SAULT STE MARIE, MI 786 786 786 786 873 936 1053 1176 1182 1014 1173 1185 1194 1200 1164 1179 1191 927 1128 1188 1197
MI146 TRAVERSE CITY, MI 1143 1143 1143 1143 1287 1383 1530 1686 1692 1479 1683 1695 1713 1824 1674 1689 1701 1371 1626 1698 1803
MI148 GRAND HAVEN, MI 1062 1062 1062 1062 1239 1341 1416 1548 1623 1398 1545 1632 1758 1779 1488 1608 1752 1338 1467 1659 1770
MI152 BATTLE CREEK/KALAMAZOO, MI 903 903 903 903 969 1239 1266 1344 1461 1254 1341 1473 1662 1698 1275 1437 1650 1011 1269 1512 1686
MI153 LANSING, MI 990 990 990 990 1098 1278 1311 1503 1578 1293 1500 1584 1713 1752 1446 1560 1701 1161 1401 1611 1740
MI154 GRAND RAPIDS, MI 1110 1110 1110 1110 1245 1314 1488 1668 1674 1422 1665 1677 1686 1692 1659 1671 1683 1299 1599 1680 1689
MI155 ANN ARBOR, MI 1467 1467 1467 1467 1548 1755 1797 2028 2145 1770 2025 2157 2349 2403 1947 2121 2337 1599 1890 2199 2391
MI156 SAGINAW, MI 759 759 759 759 861 951 1017 1176 1209 972 1173 1212 1269 1278 1146 1203 1266 876 1104 1224 1272
MN158 DULUTH, MN 1290 1290 1290 1290 1368 1428 1551 1692 1707 1509 1689 1710 1755 1794 1668 1704 1743 1416 1629 1719 1782
MN159 MINNEAPOLIS/ST PAUL, MN 1302 1302 1302 1302 1425 1545 1620 1803 1899 1581 1800 1908 2067 2112 1734 1878 2055 1494 1698 1941 2100
MO160 KANSAS CITY, MO 1197 1197 1197 1197 1365 1461 1515 1578 1584 1506 1575 1587 1596 1602 1563 1581 1593 1458 1551 1590 1599
MO161 ST.LOUIS, MO 1224 1224 1224 1224 1392 1491 1506 1590 1731 1503 1587 1746 1962 1971 1512 1701 1959 1488 1509 1794 1965
MO162 WHITEMAN AFB, MO 735 735 735 735 819 957 1038 1131 1230 969 1128 1233 1299 1410 1086 1170 1272 873 1053 1236 1389
MO163 FORT LEONARD WOOD, MO 690 690 690 690 768 822 888 984 1014 873 981 1017 1092 1170 954 1008 1071 819 933 1032 1152
MO164 SPRINGFIELD, MO 795 795 795 795 867 915 969 1047 1077 960 1044 1080 1146 1215 1017 1071 1131 912 1005 1092 1212
MO165 COLUMBIA/JEFFERSON CITY,MO 810 810 810 810 840 867 960 1149 1302 909 1038 1305 1323 1386 999 1239 1320 864 963 1308 1383
MO344 SAINT JOSEPH, MO 717 717 717 717 795 984 999 1104 1179 996 1101 1188 1311 1317 1044 1164 1308 867 1017 1215 1314
MS168 GULFPORT, MS 969 969 969 969 1062 1125 1221 1323 1329 1194 1320 1332 1347 1407 1311 1326 1338 1119 1281 1335 1392
MS169 COLUMBUS AFB, MS 741 741 741 741 861 951 996 1047 1134 966 1017 1137 1284 1341 1008 1086 1266 933 999 1149 1326
MS170 JACKSON, MS 1074 1074 1074 1074 1197 1275 1386 1536 1581 1350 1533 1587 1674 1710 1494 1572 1665 1266 1461 1605 1698
MS171 MERIDIAN, MS 747 747 747 747 828 906 978 1059 1176 930 1041 1179 1227 1341 996 1098 1212 879 981 1182 1338
MS172 HATTIESBURG, MS 1032 1032 1032 1032 1089 1128 1167 1230 1257 1164 1227 1260 1332 1407 1200 1251 1311 1125 1191 1272 1392
MT175 MALMSTROM AFB/GREAT FLS, MT 756 756 756 756 825 873 960 1062 1152 924 1029 1155 1182 1308 996 1110 1167 870 978 1158 1284
MT347 HELENA, MT 774 774 774 774 855 1101 1149 1197 1269 1113 1167 1281 1482 1542 1158 1242 1464 906 1152 1323 1527
NC176 OUTER BANKS, NC 1257 1257 1257 1257 1410 1590 1674 1800 1851 1653 1797 1857 1950 1986 1755 1839 1941 1587 1731 1875 1974
NC177 MOREHEAD/CHERRY PT MCAS, NC 1158 1158 1158 1158 1245 1302 1320 1332 1338 1317 1329 1341 1350 1395 1326 1335 1347 1299 1323 1344 1380
NC178 CAMP LEJEUNE, NC 1098 1098 1098 1098 1197 1260 1278 1290 1296 1275 1287 1299 1308 1407 1284 1293 1305 1257 1281 1302 1386
NC179 CHARLOTTE, NC 1221 1221 1221 1221 1386 1485 1578 1695 1713 1551 1692 1716 1758 1764 1668 1710 1755 1479 1641 1725 1761
NC180 DURHAM/CHAPEL HILL, NC 1245 1245 1245 1245 1365 1497 1536 1707 1815 1509 1704 1827 1995 2013 1632 1791 1992 1434 1602 1863 2004
NC181 ELIZABETH CITY, NC 1248 1248 1248 1248 1407 1500 1521 1620 1758 1518 1617 1773 1983 1989 1527 1728 1980 1497 1524 1818 1986
NC182 FORT BRAGG/POPE, NC 1017 1017 1017 1017 1098 1239 1254 1386 1485 1251 1383 1494 1650 1656 1314 1464 1647 1149 1290 1530 1653
NC183 SEYMOUR JOHNSON AFB, NC 1098 1098 1098 1098 1101 1107 1113 1125 1206 1110 1122 1209 1233 1353 1119 1152 1230 1104 1116 1212 1329
NC184 GREENSBORO, NC 975 975 975 975 1089 1161 1197 1290 1359 1194 1287 1368 1500 1593 1236 1344 1476 1158 1227 1392 1575
NC185 RALEIGH, NC 1212 1212 1212 1212 1347 1425 1509 1644 1722 1488 1641 1731 1872 1947 1587 1704 1851 1422 1563 1758 1929
NC186 WILMINGTON, NC 1122 1122 1122 1122 1260 1341 1374 1473 1575 1371 1470 1587 1749 1758 1398 1554 1746 1338 1389 1623 1752
NC187 ASHEVILLE, NC 1182 1182 1182 1182 1290 1365 1482 1644 1701 1443 1641 1707 1812 1857 1596 1689 1800 1356 1560 1728 1845
ND188 BISMARCK, ND 897 897 897 897 1005 1239 1281 1347 1461 1254 1344 1473 1665 1719 1290 1437 1650 1071 1284 1515 1704
ND189 FARGO, ND 798 798 798 798 894 1209 1224 1278 1404 1221 1275 1416 1611 1617 1230 1377 1608 939 1227 1458 1614
ND190 GRAND FORKS, ND 918 918 918 918 1029 1146 1230 1392 1440 1188 1389 1446 1530 1548 1353 1431 1524 1095 1311 1464 1539
ND191 MINOT AFB, ND 900 900 900 900 972 1287 1305 1329 1476 1302 1326 1491 1716 1722 1311 1443 1713 1017 1308 1539 1719
NE192 OMAHA/OFFUTT AFB, NE 942 942 942 942 1071 1323 1341 1500 1599 1338 1497 1608 1770 1791 1428 1578 1761 1104 1371 1644 1782
NE193 LINCOLN, NE 885 885 885 885 966 1149 1164 1203 1326 1161 1200 1341 1533 1545 1170 1302 1527 1017 1167 1383 1536
NH194 PORTSMOUTH, NH/KITTERY, ME 1536 1536 1536 1536 1689 1848 2025 2301 2358 1935 2298 2364 2511 2724 2253 2346 2460 1773 2175 2385 2688
NH195 MANCHESTER/CONCORD, NH 1539 1539 1539 1539 1749 1914 2034 2265 2373 1977 2262 2382 2571 2658 2190 2349 2547 1863 2136 2421 2640
NJ196 ATLANTIC CITY, NJ 1416 1416 1416 1416 1626 1761 1824 1992 2127 1803 1989 2139 2358 2418 1899 2097 2343 1740 1875 2184 2403
NJ198 CAPE MAY, NJ 1182 1182 1182 1182 1314 1455 1578 1740 1782 1536 1737 1788 1884 1971 1701 1773 1863 1443 1659 1806 1953
NJ200 FORT MONMOUTH/EARLE NWS, NJ 1875 1875 1875 1875 2166 2322 2376 2478 2535 2367 2475 2544 2667 2796 2433 2523 2637 2319 2415 2565 2772
NJ201 PERTH AMBOY, NJ 1569 1569 1569 1569 1800 1995 2094 2313 2442 2052 2310 2454 2664 2700 2220 2415 2655 1956 2178 2499 2688
NJ202 NORTHERN NEW JERSEY 2112 2112 2112 2112 2412 2637 2823 3054 3111 2751 3051 3117 3225 3282 3006 3099 3210 2616 2943 3138 3270
NJ203 TRENTON, NJ 1629 1629 1629 1629 1830 1965 2184 2448 2511 2097 2445 2517 2628 2670 2400 2496 2616 1941 2325 2541 2658
NJ204 JB MCGUIRE-DIX-LAKEHURST, NJ 1473 1473 1473 1473 1668 1779 1872 2028 2121 1845 2025 2130 2289 2349 1962 2100 2274 1776 1932 2163 2334
NM205 HOLLOMAN AFB/ALAMOGORDO, NM 852 852 852 852 924 1080 1104 1278 1335 1095 1275 1341 1446 1479 1230 1323 1437 951 1185 1365 1470
NM206 ALBUQUERQUE/KIRTLAND AFB, NM 1104 1104 1104 1104 1269 1362 1458 1557 1563 1431 1554 1566 1575 1581 1545 1560 1572 1359 1518 1569 1578
NM207 CANNON AFB/CLOVIS, NM 693 693 693 693 765 972 1017 1080 1161 987 1077 1167 1311 1368 1029 1143 1293 822 1020 1197 1356
NM209 WHITE SANDS MR/LAS CRUCES, NM 780 780 780 780 918 987 1047 1248 1269 999 1245 1272 1314 1329 1224 1266 1311 933 1164 1281 1320
NM210 SANTA FE/LOS ALAMOS, NM 1272 1272 1272 1272 1413 1503 1632 1800 1917 1590 1797 1920 1992 2187 1758 1842 1950 1494 1716 1923 2154
NV211 FALLON NAS, NV 885 885 885 885 990 1065 1185 1347 1395 1134 1344 1398 1455 1605 1320 1365 1416 1032 1275 1401 1578
NV212 NELLIS AFB/LAS VEGAS, NV 1272 1272 1272 1272 1419 1503 1590 1695 1713 1566 1692 1716 1758 1785 1674 1710 1749 1500 1647 1722 1773
NV213 RENO/CARSON CITY, NV 1428 1428 1428 1428 1602 1701 1782 1911 1980 1761 1908 1986 2100 2112 1857 1965 2097 1698 1833 2013 2103
NY215 BALLSTON SPA/ALBANY, NY 1584 1584 1584 1584 1809 1938 2043 2163 2172 2013 2160 2175 2208 2271 2145 2169 2190 1932 2112 2178 2256
NY216 BUFFALO, NY 1407 1407 1407 1407 1554 1695 1884 2082 2088 1809 2079 2091 2100 2151 2070 2085 2097 1674 2007 2094 2136
NY217 WEST POINT, NY 1590 1590 1590 1590 1905 2094 2118 2283 2478 2115 2280 2496 2793 2811 2157 2436 2787 2076 2145 2562 2802
NY218 LONG ISLAND, NY 2679 2679 2679 2679 2967 3141 3369 3663 3753 3279 3660 3765 3939 4053 3594 3735 3909 3117 3516 3798 4032
NY219 NEW YORK CITY, NY 2700 2700 2700 2700 2817 2907 3102 3336 3384 3027 3333 3390 3537 3801 3294 3375 3468 2886 3228 3408 3756
NY221 ROCHESTER, NY 1080 1080 1080 1080 1212 1329 1449 1653 1695 1383 1650 1698 1779 1815 1617 1686 1767 1260 1560 1716 1806
NY222 ROME/GRIFFISS AFB, NY 1041 1041 1041 1041 1212 1551 1602 1710 1845 1563 1707 1857 2082 2151 1617 1815 2064 1254 1605 1905 2133
NY223 SYRACUSE, NY 1077 1077 1077 1077 1266 1329 1446 1713 1731 1347 1710 1734 1794 1884 1689 1725 1770 1278 1605 1743 1866
NY225 FORT DRUM/WATERTOWN, NY 987 987 987 987 1098 1164 1299 1482 1617 1227 1374 1620 1629 1785 1338 1569 1626 1161 1302 1623 1749
NY349 WESTCHESTER COUNTY, NY 2484 2484 2484 2484 3003 3279 3312 3546 3864 3309 3543 3894 4368 4374 3342 3795 4365 3276 3336 3999 4371
NY413 STATEN ISLAND, NY 2346 2346 2346 2346 2619 2772 2901 3036 3042 2859 3033 3045 3087 3297 3024 3039 3051 2763 2985 3048 3264
OH227 AKRON, OH 1047 1047 1047 1047 1107 1278 1296 1323 1467 1293 1320 1482 1707 1734 1302 1437 1701 1149 1299 1530 1722
OH228 CINCINNATI, OH 1236 1236 1236 1236 1425 1554 1650 1806 1890 1623 1803 1899 2037 2043 1743 1872 2034 1548 1713 1932 2040
OH229 CLEVELAND, OH 1164 1164 1164 1164 1293 1371 1398 1470 1542 1395 1467 1548 1674 1704 1413 1527 1665 1368 1410 1575 1695
OH230 COLUMBUS, OH 1029 1029 1029 1029 1158 1290 1344 1461 1557 1302 1458 1566 1734 1809 1392 1536 1716 1233 1368 1602 1791
OH231 WRIGHT-PATTERSON AFB, OH 936 936 936 936 1020 1080 1131 1239 1311 1122 1236 1320 1449 1497 1182 1296 1437 1074 1167 1347 1485
OH232 TOLEDO, OH 1083 1083 1083 1083 1281 1395 1443 1518 1548 1434 1515 1551 1608 1620 1488 1542 1605 1392 1476 1563 1611
OH233 YOUNGSTOWN, OH 693 693 693 693 798 888 930 1095 1125 900 1092 1128 1188 1218 1065 1119 1179 816 1020 1140 1206
OK235 ALTUS AFB, OK 693 693 693 693 744 852 918 1023 1077 867 1020 1080 1158 1245 987 1053 1134 777 954 1083 1224
OK236 VANCE AFB/ENID, OK 753 753 753 753 813 936 984 1122 1164 951 1119 1170 1260 1371 1083 1155 1245 852 1047 1188 1368
OK237 FORT SILL/LAWTON, OK 678 678 678 678 762 906 942 1011 1083 921 1008 1089 1218 1263 954 1068 1206 813 945 1116 1251
OK239 OKLAHOMA CITY, OK 939 939 939 939 1041 1158 1209 1359 1425 1179 1356 1431 1545 1572 1308 1410 1539 1101 1275 1458 1560
OK240 TULSA, OK 903 903 903 903 1008 1074 1119 1188 1206 1113 1185 1209 1260 1311 1164 1203 1248 1071 1152 1218 1302
OR241 ASTORIA, OR 1044 1044 1044 1044 1158 1389 1422 1608 1698 1401 1605 1707 1860 1905 1542 1677 1848 1239 1494 1740 1890
OR242 COOS BAY, OR 1017 1017 1017 1017 1155 1353 1428 1533 1635 1365 1530 1644 1824 1923 1461 1611 1800 1248 1431 1680 1905
OR243 PORTLAND, OR 1821 1821 1821 1821 2214 2427 2442 2454 2460 2439 2451 2463 2472 2502 2448 2457 2469 2424 2445 2466 2487
OR244 SALEM, OR 1137 1137 1137 1137 1260 1338 1431 1578 1653 1401 1575 1662 1794 1851 1521 1638 1779 1329 1494 1689 1839
OR245 CORVALLIS, OR 1200 1200 1200 1200 1341 1635 1689 1869 1986 1647 1866 1998 2193 2271 1788 1962 2175 1407 1728 2037 2253
OR246 EUGENE, OR 1419 1419 1419 1419 1566 1659 1764 1890 1911 1731 1887 1914 1968 2016 1866 1908 1956 1650 1833 1926 2001
PA247 CARLISLE BARRACKS, PA 1122 1122 1122 1122 1275 1371 1488 1650 1713 1449 1647 1719 1839 1902 1599 1701 1821 1362 1563 1743 1887
PA248 PHILADELPHIA, PA/CAMDEN, NJ 1386 1386 1386 1386 1578 1725 1854 2067 2154 1800 2064 2163 2304 2319 2001 2136 2298 1689 1953 2193 2310
PA249 WILLOW GROVE, PA 1521 1521 1521 1521 1698 1854 1995 2241 2325 1926 2238 2334 2478 2508 2175 2307 2469 1794 2115 2364 2499
PA250 PITTSBURGH, PA 1383 1383 1383 1383 1491 1695 1743 1983 2088 1707 1980 2097 2259 2265 1911 2064 2256 1554 1854 2133 2262
PA252 STATE COLLEGE, PA 1035 1035 1035 1035 1098 1311 1329 1497 1590 1326 1494 1602 1752 1779 1431 1572 1746 1140 1383 1632 1767
PA253 ERIE, PA 1029 1029 1029 1029 1125 1188 1245 1356 1425 1233 1353 1434 1551 1563 1299 1410 1548 1185 1284 1461 1554
PA254 WILKES-BARRE/SCRANTON, PA 1134 1134 1134 1134 1176 1410 1428 1608 1710 1425 1605 1719 1881 1887 1533 1686 1878 1209 1482 1755 1884
PA255 ALLENTOWN/BETHLEHEM, PA 1503 1503 1503 1503 1665 1758 1842 1950 1980 1821 1947 1986 2052 2085 1917 1974 2043 1755 1893 1998 2073
RI256 NEWPORT, RI 1461 1461 1461 1461 1620 2001 2034 2262 2418 2013 2259 2433 2676 2721 2160 2382 2664 1704 2085 2484 2709
RI257 PROVIDENCE, RI 1401 1401 1401 1401 1587 1716 1875 2049 2058 1818 2046 2061 2103 2199 2034 2055 2079 1701 1980 2064 2178
SC258 BEAUFORT/PARRIS ISLAND, SC 1251 1251 1251 1251 1458 1575 1662 1776 1803 1638 1773 1806 1890 2028 1749 1797 1854 1569 1722 1818 2001
SC259 CHARLESTON, SC 1365 1365 1365 1365 1563 1674 1752 1851 1869 1734 1848 1872 1914 1938 1830 1866 1908 1671 1806 1881 1929
SC260 COLUMBIA/FORT JACKSON, SC 1233 1233 1233 1233 1368 1449 1491 1545 1551 1485 1542 1554 1575 1653 1530 1548 1560 1446 1521 1557 1635
SC261 GREENVILLE, SC 1113 1113 1113 1113 1245 1320 1359 1440 1488 1356 1437 1494 1584 1596 1395 1479 1578 1317 1386 1515 1587
SC262 MYRTLE BEACH, SC 1188 1188 1188 1188 1275 1335 1437 1557 1575 1407 1554 1578 1644 1749 1533 1572 1617 1329 1500 1587 1728
SC263 SUMTER/SHAW AFB, SC 876 876 876 876 921 966 1014 1107 1173 999 1104 1179 1302 1365 1056 1158 1284 951 1041 1203 1350
SD264 RAPID CITY/ELLSWORTH AFB, SD 924 924 924 924 1032 1167 1230 1419 1467 1182 1416 1473 1560 1584 1377 1458 1554 1068 1329 1491 1572
SD265 SIOUX FALLS, SD 843 843 843 843 915 1089 1110 1233 1314 1104 1230 1323 1455 1485 1173 1296 1449 963 1140 1350 1473
TN266 CHATTANOOGA, TN 1074 1074 1074 1074 1269 1377 1401 1485 1602 1398 1482 1614 1797 1818 1407 1575 1791 1374 1404 1653 1809
TN267 KNOXVILLE, TN 1038 1038 1038 1038 1128 1185 1272 1398 1458 1248 1395 1464 1584 1683 1353 1446 1557 1182 1326 1485 1662
TN268 MEMPHIS, TN 1371 1371 1371 1371 1551 1653 1743 1860 1893 1719 1857 1899 1962 1971 1824 1887 1959 1650 1800 1914 1965
TN269 NASHVILLE, TN 1440 1440 1440 1440 1725 1878 1905 1953 1995 1902 1950 2001 2073 2088 1917 1986 2070 1875 1914 2016 2079
TN353 JOHNSON CITY/KINGSPORT, TN 870 870 870 870 903 951 993 1077 1146 972 1074 1152 1278 1359 1023 1131 1263 930 1011 1179 1356
TX270 ABILENE/DYESS AFB, TX 891 891 891 891 984 1131 1170 1296 1374 1146 1293 1383 1518 1569 1236 1359 1506 1041 1206 1410 1557
TX272 AUSTIN, TX 1431 1431 1431 1431 1656 1923 2046 2178 2298 1938 2133 2316 2601 2766 2073 2265 2559 1782 2049 2370 2736
TX273 BEAUMONT, TX 1065 1065 1065 1065 1221 1317 1422 1557 1593 1389 1554 1599 1674 1716 1524 1587 1662 1308 1491 1614 1704
TX274 COLLEGE STATION, TX 1077 1077 1077 1077 1254 1353 1410 1494 1515 1401 1491 1518 1575 1626 1467 1509 1560 1350 1452 1527 1611
TX275 CORPUS CHRISTI, TX 1122 1122 1122 1122 1263 1398 1497 1686 1749 1446 1683 1758 1875 1929 1632 1737 1860 1344 1587 1782 1917
TX277 DALLAS, TX 1302 1302 1302 1302 1449 1932 1947 2061 2256 1944 2058 2277 2574 2580 1953 2214 2571 1539 1950 2343 2577
TX278 LAUGHLIN AFB/DEL RIO, TX 810 810 810 810 921 1137 1182 1290 1374 1149 1287 1383 1527 1587 1227 1356 1512 951 1185 1413 1572
TX279 EL PASO, TX 987 987 987 987 1110 1185 1254 1365 1422 1236 1362 1428 1524 1530 1320 1410 1521 1182 1299 1449 1527
TX281 BROWNSVILLE, TX 924 924 924 924 1056 1176 1218 1272 1383 1188 1269 1395 1578 1635 1227 1359 1563 1128 1221 1431 1620
TX282 HOUSTON, TX 1203 1203 1203 1203 1410 1536 1605 1731 1809 1590 1728 1818 1947 1959 1671 1791 1941 1533 1650 1845 1950
TX283 LUBBOCK, TX 846 846 846 846 912 1185 1200 1212 1335 1197 1209 1350 1578 1584 1206 1302 1575 954 1203 1401 1581
TX284 GOODFELLOW AFB, TX 888 888 888 888 993 1194 1239 1380 1458 1209 1377 1467 1605 1662 1323 1443 1590 1038 1278 1494 1647
TX285 SAN ANTONIO, TX 1239 1239 1239 1239 1434 1548 1659 1773 1779 1623 1770 1782 1791 1797 1761 1776 1788 1542 1728 1785 1794
TX286 FORT HOOD, TX 945 945 945 945 1065 1137 1155 1179 1224 1152 1176 1230 1329 1428 1161 1215 1305 1134 1158 1248 1410
TX288 WICHITA FLS/SHEPPARD AFB, TX 810 810 810 810 936 1011 1050 1146 1275 1038 1107 1278 1323 1464 1068 1188 1320 1008 1056 1281 1461
TX356 FORT WORTH, TX 1269 1269 1269 1269 1425 1515 1587 1692 1734 1569 1689 1740 1827 1878 1653 1725 1812 1512 1632 1758 1863
TX415 WACO,TX 954 954 954 954 1092 1173 1221 1302 1338 1212 1299 1341 1437 1560 1266 1329 1407 1170 1254 1356 1536
UT291 OGDEN/HILL AFB, UT 1203 1203 1203 1203 1335 1419 1512 1608 1614 1485 1605 1617 1626 1689 1599 1611 1623 1413 1572 1620 1674
UT292 SALT LAKE CITY, UT 1143 1143 1143 1143 1272 1353 1455 1581 1602 1425 1578 1605 1686 1827 1554 1599 1650 1347 1521 1617 1800
UT357 PROVO, UT 1314 1314 1314 1314 1347 1383 1497 1617 1623 1461 1614 1626 1635 1695 1608 1620 1632 1377 1572 1629 1680
VA295 CHARLOTTESVILLE, VA 1419 1419 1419 1419 1554 1638 1743 1908 1986 1710 1905 1995 2157 2310 1848 1968 2118 1629 1815 2022 2283
VA296 QUANTICO/WOODBRIDGE, VA 1551 1551 1551 1551 1650 1713 1758 1818 1824 1752 1815 1827 1863 1998 1806 1821 1833 1710 1791 1830 1974
VA297 HAMPTON/NEWPORT NEWS, VA 1170 1170 1170 1170 1335 1428 1473 1533 1539 1467 1530 1542 1557 1569 1518 1536 1554 1425 1506 1545 1560
VA298 NORFOLK/PORTSMOUTH, VA 1365 1365 1365 1365 1602 1731 1746 1770 1809 1743 1767 1815 1890 1935 1752 1800 1878 1728 1749 1830 1923
VA301 RICHMOND/FORT LEE, VA 1131 1131 1131 1131 1305 1404 1422 1473 1551 1419 1470 1560 1686 1695 1428 1533 1683 1401 1425 1587 1689
VA302 WARRENTON, VA 1653 1653 1653 1653 1887 2022 2154 2310 2328 2109 2307 2331 2379 2418 2286 2325 2370 2010 2241 2340 2406
VA303 LEXINGTON, VA 906 906 906 906 987 1083 1098 1170 1272 1095 1167 1284 1443 1452 1104 1251 1440 1035 1101 1317 1446
VA362 ROANOKE, VA 987 987 987 987 1089 1155 1260 1401 1446 1227 1398 1452 1548 1629 1362 1437 1527 1146 1329 1470 1611
VA368 DAHLGREN/FORT AP HILL, VA 1470 1470 1470 1470 1617 1704 1767 1854 1875 1755 1851 1878 1926 1941 1827 1872 1920 1701 1809 1890 1932
VT305 BURLINGTON, VT 1545 1545 1545 1545 1698 1905 1989 2109 2271 1917 2106 2289 2565 2679 2010 2238 2535 1782 1992 2343 2655
WA306 BREMERTON, WA 1500 1500 1500 1500 1647 1731 1812 1944 2010 1791 1941 2016 2145 2223 1890 1995 2124 1728 1866 2043 2205
WA307 EVERETT, WA 1695 1695 1695 1695 1878 1992 2142 2319 2355 2088 2316 2358 2436 2493 2286 2346 2418 1977 2238 2373 2478
WA308 PORT ANGELES, WA 1059 1059 1059 1059 1188 1329 1419 1599 1662 1371 1596 1668 1791 1875 1548 1647 1770 1272 1506 1692 1857
WA309 SEATTLE, WA 1905 1905 1905 1905 2184 2409 2517 2790 2952 2454 2787 2967 3222 3255 2679 2916 3210 2334 2625 3021 3246
WA310 SPOKANE, WA 957 957 957 957 1077 1239 1305 1485 1545 1251 1482 1551 1671 1761 1437 1533 1647 1110 1386 1575 1740
WA311 TACOMA, WA 1518 1518 1518 1518 1695 1800 1914 2064 2115 1878 2061 2121 2220 2277 2022 2106 2205 1791 1986 2142 2262
WA312 WHIDBEY ISLAND, WA 1128 1128 1128 1128 1281 1395 1449 1557 1668 1413 1554 1680 1875 1947 1476 1644 1857 1365 1461 1722 1929
WA313 YAKIMA, WA 918 918 918 918 1032 1266 1347 1443 1536 1281 1440 1545 1716 1821 1377 1515 1686 1071 1350 1578 1800
WI316 MADISON, WI 1191 1191 1191 1191 1323 1455 1593 1743 1791 1545 1740 1794 1815 2034 1728 1746 1800 1443 1683 1797 1998
WI317 MILWAUKEE, WI 1437 1437 1437 1437 1635 1767 1923 2091 2097 1866 2088 2100 2109 2115 2079 2094 2106 1752 2028 2103 2112
WI318 SPARTA/FORT MCCOY, WI 942 942 942 942 1071 1146 1182 1260 1320 1179 1257 1326 1428 1461 1212 1305 1419 1143 1206 1347 1452
WI359 STEVENS POINT, WI 750 750 750 750 813 1011 1026 1134 1212 1023 1131 1221 1347 1368 1074 1194 1344 852 1041 1248 1359
WV320 MORGANTOWN, WV 939 939 939 939 1050 1167 1257 1434 1476 1206 1431 1482 1560 1596 1398 1467 1551 1098 1350 1497 1584
WV322 HUNTINGTON, WV 999 999 999 999 1143 1224 1281 1380 1428 1272 1377 1431 1527 1590 1338 1416 1512 1221 1323 1452 1575
WV323 CHARLESTON, WV 777 777 777 777 834 879 987 1113 1131 954 1110 1134 1170 1176 1092 1128 1167 873 1056 1140 1173
WV454 EASTERN PANHANDLE, WV 1119 1119 1119 1119 1248 1332 1452 1584 1593 1413 1581 1596 1638 1749 1569 1590 1611 1323 1530 1599 1728
WY324 CHEYENNE, WY 780 780 780 780 894 1134 1203 1278 1353 1149 1257 1362 1533 1626 1221 1332 1509 924 1206 1395 1605
ZZ510 COUNTY COST GROUP 510 798 798 798 798 885 957 1014 1125 1179 996 1122 1185 1287 1338 1083 1167 1272 939 1062 1206 1323
ZZ520 COUNTY COST GROUP 520 804 804 804 804 891 963 1020 1134 1188 1005 1131 1194 1293 1347 1089 1176 1281 945 1068 1215 1332
ZZ530 COUNTY COST GROUP 530 810 810 810 810 900 972 1029 1143 1197 1011 1140 1203 1302 1356 1098 1185 1290 954 1077 1221 1341
ZZ540 COUNTY COST GROUP 540 816 816 816 816 906 978 1038 1152 1203 1020 1149 1212 1311 1365 1107 1194 1299 960 1086 1230 1350
ZZ550 COUNTY COST GROUP 550 822 822 822 822 912 984 1044 1161 1212 1026 1158 1218 1320 1374 1116 1203 1308 969 1092 1239 1359
ZZ560 COUNTY COST GROUP 560 828 828 828 828 918 990 1053 1167 1221 1035 1164 1227 1329 1383 1122 1209 1314 975 1101 1248 1368
ZZ570 COUNTY COST GROUP 570 834 834 834 834 924 996 1059 1176 1227 1041 1173 1236 1338 1389 1131 1218 1323 981 1107 1254 1377
ZZ580 COUNTY COST GROUP 580 840 840 840 840 933 1002 1065 1182 1236 1047 1179 1242 1344 1398 1137 1224 1332 987 1116 1263 1383
ZZ590 COUNTY COST GROUP 590 843 843 843 843 936 1008 1071 1188 1242 1053 1185 1248 1353 1404 1143 1233 1338 993 1122 1269 1392
ZZ600 COUNTY COST GROUP 600 849 849 849 849 942 1011 1077 1194 1248 1059 1191 1254 1359 1410 1149 1239 1344 999 1128 1275 1398
ZZ610 COUNTY COST GROUP 610 855 855 855 855 948 1017 1086 1203 1257 1068 1200 1263 1368 1419 1158 1245 1353 1005 1137 1284 1407
ZZ620 COUNTY COST GROUP 620 861 861 861 861 957 1026 1092 1212 1266 1074 1209 1272 1377 1428 1167 1254 1362 1014 1143 1293 1416
ZZ630 COUNTY COST GROUP 630 867 867 867 867 963 1032 1101 1221 1275 1083 1218 1281 1386 1440 1176 1263 1371 1023 1152 1302 1425
ZZ640 COUNTY COST GROUP 640 873 873 873 873 972 1038 1110 1230 1284 1092 1227 1290 1395 1449 1185 1272 1380 1029 1161 1311 1434
ZZ650 COUNTY COST GROUP 650 879 879 879 879 978 1047 1119 1239 1293 1098 1236 1299 1404 1458 1194 1281 1392 1038 1170 1320 1443
ZZ660 COUNTY COST GROUP 660 888 888 888 888 987 1053 1128 1248 1305 1107 1245 1311 1416 1467 1203 1293 1401 1047 1179 1332 1455
ZZ670 COUNTY COST GROUP 670 894 894 894 894 993 1062 1137 1257 1314 1116 1254 1320 1425 1479 1212 1302 1410 1053 1188 1341 1464
ZZ680 COUNTY COST GROUP 680 900 900 900 900 1002 1068 1146 1266 1323 1125 1263 1329 1434 1488 1221 1311 1419 1062 1197 1350 1473
ZZ690 COUNTY COST GROUP 690 906 906 906 906 1008 1074 1152 1275 1329 1131 1272 1335 1443 1497 1227 1317 1428 1068 1203 1359 1482
ZZ700 COUNTY COST GROUP 700 909 909 909 909 1014 1080 1158 1281 1335 1137 1278 1344 1449 1503 1233 1323 1434 1074 1209 1365 1488
ZZ710 COUNTY COST GROUP 710 915 915 915 915 1020 1086 1164 1287 1344 1143 1284 1350 1455 1509 1242 1332 1443 1080 1218 1371 1497
ZZ720 COUNTY COST GROUP 720 924 924 924 924 1029 1092 1173 1299 1353 1152 1296 1362 1467 1521 1251 1341 1452 1089 1227 1383 1506
ZZ730 COUNTY COST GROUP 730 933 933 933 933 1038 1104 1185 1308 1365 1164 1305 1371 1479 1533 1263 1353 1464 1098 1239 1395 1518
ZZ740 COUNTY COST GROUP 740 942 942 942 942 1050 1116 1200 1323 1380 1176 1320 1389 1494 1548 1278 1368 1482 1113 1254 1410 1536
ZZ750 COUNTY COST GROUP 750 960 960 960 960 1071 1140 1224 1350 1407 1200 1347 1413 1521 1578 1302 1395 1509 1137 1278 1437 1563
ZZ760 COUNTY COST GROUP 760 978 978 978 978 1092 1161 1245 1374 1431 1224 1371 1440 1548 1602 1326 1419 1533 1158 1302 1461 1587
ZZ770 COUNTY COST GROUP 770 999 999 999 999 1113 1185 1269 1401 1458 1248 1398 1464 1575 1629 1353 1446 1560 1179 1326 1488 1617
ZZ780 COUNTY COST GROUP 780 1020 1020 1020 1020 1140 1212 1299 1431 1491 1275 1428 1497 1608 1665 1383 1479 1593 1206 1356 1521 1650
ZZ790 COUNTY COST GROUP 790 1038 1038 1038 1038 1161 1233 1323 1458 1515 1299 1455 1524 1635 1689 1407 1503 1620 1230 1380 1545 1677
ZZ800 COUNTY COST GROUP 800 1065 1065 1065 1065 1191 1266 1359 1494 1554 1332 1491 1560 1674 1731 1446 1542 1659 1263 1416 1584 1716
ZZ810 COUNTY COST GROUP 810 1092 1092 1092 1092 1221 1299 1392 1530 1590 1365 1527 1599 1713 1770 1482 1578 1698 1293 1452 1620 1755
ZZ820 COUNTY COST GROUP 820 1128 1128 1128 1128 1260 1341 1437 1578 1641 1410 1575 1647 1764 1821 1530 1629 1749 1338 1500 1671 1806
ZZ830 COUNTY COST GROUP 830 1170 1170 1170 1170 1308 1392 1491 1635 1698 1461 1632 1707 1824 1881 1584 1686 1809 1386 1554 1728 1866
ZZ840 COUNTY COST GROUP 840 1206 1206 1206 1206 1350 1437 1539 1686 1752 1509 1683 1758 1878 1935 1635 1737 1863 1431 1605 1782 1923
ZZ850 COUNTY COST GROUP 850 1257 1257 1257 1257 1407 1500 1602 1755 1821 1572 1752 1827 1950 2007 1704 1806 1935 1491 1671 1851 1995
ZZ860 COUNTY COST GROUP 860 1305 1305 1305 1305 1464 1560 1665 1821 1890 1632 1818 1896 2022 2079 1770 1875 2004 1551 1737 1923 2067
ZZ870 COUNTY COST GROUP 870 1374 1374 1374 1374 1542 1641 1755 1917 1986 1719 1914 1992 2121 2181 1863 1971 2103 1635 1827 2019 2166
"""
withoutDependContstr = """
2400 2514 2556
2376 2397 2439
2652 2742 2793
2274 2409 2451
2370 2541 2586
2157 2256 2295
1215 1218 1221
1500 1623 1647
1719 1722 1731
1713 1824 1854
1356 1407 1428
1563 1710 1737
1677 1680 1689
1533 1539 1566
1302 1392 1413
1545 1557 1569
1995 2034 2070
1458 1479 1503
1722 1725 1734
1461 1491 1515
3711 3747 3816
5070 5073 5151
1515 1650 1677
1881 1950 1983
1746 1749 1758
3198 3201 3240
3033 3051 3108
2394 2397 2418
3735 3738 3789
1656 1722 1752
2790 2793 2823
1542 1617 1644
2325 2328 2349
2436 2448 2493
1995 2013 2028
2763 2766 2781
3492 3495 3549
3240 3270 3330
3513 3762 3831
2373 2379 2421
2070 2109 2145
4869 4959 5052
2949 2979 3033
1977 1980 1992
1503 1527 1551
2232 2361 2403
2481 2484 2508
2019 2022 2055
1983 2058 2094
2664 2703 2751
2034 2037 2052
2247 2295 2334
3003 3006 3039
2982 3147 3204
1836 1926 1959
1947 2049 2085
1785 1902 1932
1941 2022 2055
1944 1992 2025
2760 2814 2862
2052 2055 2070
2115 2220 2259
1767 1797 1827
1851 1869 1893
2160 2163 2181
2361 2421 2463
1569 1572 1593
3657 3837 3906
1803 1812 1842
1887 1890 1914
1851 1920 1953
2388 2391 2412
1443 1455 1464
1608 1626 1650
1458 1467 1488
1563 1566 1572
1563 1572 1584
1758 1761 1770
2079 2082 2097
1629 1689 1716
1356 1419 1443
3588 3702 3771
3315 3498 3561
2445 2448 2484
3003 3006 3042
1791 1848 1878
1680 1707 1737
1851 1860 1893
1407 1416 1425
2058 2076 2097
1740 1752 1782
2301 2391 2433
1599 1623 1647
2304 2316 2358
1521 1524 1527
1551 1569 1596
1647 1662 1689
1137 1143 1149
1863 1878 1893
1398 1452 1476
1395 1407 1416
1671 1677 1704
1449 1452 1458
1668 1767 1797
1422 1425 1443
1779 1857 1887
1512 1593 1617
1413 1425 1434
1320 1329 1338
1626 1641 1653
1866 1935 1965
1344 1368 1389
1968 2010 2043
1815 1818 1830
1680 1683 1695
1488 1545 1569
1986 1989 2004
1347 1380 1401
4143 4344 4425
3756 3792 3822
2676 2703 2751
2724 2736 2784
2793 2865 2916
2868 2889 2940
2172 2250 2289
3252 3351 3411
3300 3363 3423
2130 2211 2250
2544 2547 2574
2397 2439 2484
2106 2109 2130
2553 2580 2625
2469 2475 2517
2112 2115 2133
1635 1659 1686
2244 2247 2268
2487 2907 2961
2637 2670 2718
1908 1914 1947
1662 1665 1680
2298 2316 2334
1713 1749 1779
1203 1206 1209
1869 1971 2004
1782 1788 1818
1731 1743 1761
1761 1791 1821
1695 1698 1716
2421 2466 2508
1281 1284 1287
1806 1839 1869
2127 2166 2202
1605 1608 1611
1974 1977 1995
1485 1557 1581
1197 1269 1287
1386 1395 1404
1464 1674 1701
1320 1323 1326
1428 1479 1503
1446 1458 1464
1716 1743 1770
1500 1512 1524
1497 1509 1530
1362 1482 1506
1563 1617 1641
1995 2022 2058
1425 1467 1491
1449 1548 1572
1767 1770 1782
2016 2019 2052
1992 1995 2010
1659 1662 1671
1410 1542 1566
1629 1713 1743
1974 2046 2079
1761 1764 1773
1869 1908 1941
1734 1779 1809
1620 1623 1629
1551 1560 1584
1725 1728 1737
1794 1809 1839
1548 1551 1569
2817 3024 3078
2694 2775 2823
2436 2490 2532
2004 2085 2118
2847 2970 3021
2706 2736 2784
3300 3351 3414
2682 2715 2763
2370 2424 2466
1485 1512 1536
1584 1587 1590
1389 1440 1464
1332 1335 1350
2274 2469 2511
1668 1815 1845
1788 1806 1836
2115 2118 2148
2292 2346 2388
2166 2211 2250
2814 2817 2868
4101 4212 4290
3921 4185 4260
1824 1854 1887
2175 2235 2274
1938 2007 2040
1890 2115 2151
4377 4380 4443
3393 3600 3666
1875 1890 1905
2046 2049 2064
1710 1734 1761
1836 1902 1932
1533 1551 1578
1842 1857 1869
1221 1242 1260
1311 1356 1377
1587 1599 1608
1296 1311 1332
1575 1590 1617
1497 1509 1518
1917 1953 1986
1962 2055 2091
2514 2556 2601
1869 1920 1953
2499 2520 2541
2028 2067 2100
2007 2022 2040
2322 2325 2364
2514 2538 2583
2268 2271 2292
1782 1797 1827
1566 1569 1584
1890 1893 1905
2091 2115 2151
2733 2769 2817
2394 2415 2433
2088 2220 2259
1941 1956 1989
1680 1752 1779
1659 1671 1683
1791 1893 1923
1449 1461 1470
1587 1596 1623
1488 1509 1533
1821 1827 1857
1722 1815 1845
1974 1977 1989
2091 2094 2127
1566 1578 1590
1584 1629 1653
2835 2997 3051
1791 1806 1818
1653 1686 1713
1953 1995 2028
2583 2586 2613
1605 1656 1683
1533 1536 1542
1683 1698 1728
1962 1965 1992
1587 1590 1596
1683 1734 1761
1800 1803 1806
1467 1560 1584
1644 1659 1671
1893 1935 1968
1611 1725 1755
1710 1764 1794
1884 2019 2055
1713 1767 1797
2376 2526 2571
2055 2184 2223
1644 1656 1668
1947 1986 2019
1698 1701 1707
2424 2454 2496
1455 1458 1467
1659 1734 1761
1944 1947 1974
2724 2832 2883
2250 2322 2361
2514 2565 2610
1908 1986 2019
3264 3291 3351
1794 1878 1908
2295 2346 2385
1971 2037 2070
1866 1968 2001
2133 2352 2391
2121 2139 2157
1470 1494 1518
1371 1377 1398
1602 1629 1656
1611 1668 1695
1179 1182 1185
1794 1899 1929
1662 1746 1776
1353 1398 1419
1362 1407 1428
1371 1416 1437
1380 1425 1446
1389 1434 1455
1398 1443 1464
1404 1449 1473
1413 1458 1482
1422 1467 1488
1428 1473 1497
1437 1482 1506
1446 1491 1515
1455 1500 1524
1464 1509 1533
1473 1521 1545
1485 1530 1554
1494 1539 1566
1503 1551 1575
1512 1557 1584
1518 1566 1590
1527 1572 1599
1539 1584 1611
1551 1596 1623
1566 1614 1638
1593 1641 1668
1620 1668 1695
1647 1695 1722
1680 1728 1758
1707 1755 1785
1749 1797 1827
1788 1836 1866
1839 1887 1920
1899 1950 1983
1956 2007 2040
2028 2079 2115
2100 2151 2190
2202 2253 2292
"""

withoutDepend = withoutDependstr.strip().split('\n')
withoutDependCont = withoutDependContstr.strip().split('\n')

In [416]:
withoutDependComb = {}

for line in zip(withoutDepend, withoutDependCont):
    left = line[0].split()
    if left[0].startswith("ZZ"):
        continue
    right = line[1].split()
    mha = left[0]
    left.extend(right)
    withoutDependComb[mha] = {}
    i = 0
    for cell in left[1:]:
        if cell.isnumeric():
            try:
                withoutDependComb[mha][ranks[i]] = float(cell)
            except Exception as e:
                print(e)
            i += 1

In [417]:
mhaCodes = {}
with open("BAH2021/sorted_zipmha21.txt") as f:
    for line in f:
        zipCode, mha = line.split()
        mhaCodes[zipCode] = mha

### Format Military Pay Scale

In [418]:
# TODO: Make this a webscrape instead of copy/paste
milPayScaleStr = """
PAYGRADE <2 2 3 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40
O-10* 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30
O-9* 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 16012.50 16243.80 16576.80 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30 16608.30
O-8 11329.50 11701.20 11947.50 12016.20 12323.40 12836.70 12956.40 13443.60 13584.00 14004.00 14611.80 15171.90 15546.00 15546.00 15546.00 15546.00 15935.40 15935.40 16333.20 16333.20 16333.20 16333.20
O-7 9414.30 9851.40 10053.90 10215.00 10506.00 10794.00 11126.70 11458.20 11791.20 12836.70 13719.30 13719.30 13719.30 13719.30 13789.80 13789.80 14065.80 14065.80 14065.80 14065.80 14065.80 14065.80
O-6** 7139.10 7842.90 8357.70 8357.70 8389.80 8749.20 8796.90 8796.90 9296.70 10180.50 10699.20 11217.60 11512.80 11811.90 12390.90 12390.90 12638.40 12638.40 12638.40 12638.40 12638.40 12638.40
O-5 5951.40 6704.40 7168.20 7255.50 7545.60 7718.40 8099.40 8379.60 8740.80 9293.10 9555.90 9816.00 10111.20 10111.20 10111.20 10111.20 10111.20 10111.20 10111.20 10111.20 10111.20 10111.20
O-4 5135.10 5943.90 6341.10 6429.00 6797.10 7192.20 7684.20 8066.70 8332.50 8485.50 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70 8573.70
O-3*** 4514.70 5117.70 5523.30 6022.80 6311.70 6628.20 6832.80 7169.40 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20 7345.20
O-2*** 3901.20 4442.70 5116.80 5289.90 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50 5398.50
O-1*** 3385.80 3524.40 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60 4260.60
O-3E 0.00 0.00 0.00 6022.80 6311.70 6628.20 6832.80 7169.40 7453.50 7617.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00 7839.00
O-2E 0.00 0.00 0.00 5289.90 5398.50 5570.40 5860.50 6084.90 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70 6251.70
O-1E 0.00 0.00 0.00 4260.60 4549.50 4717.50 4889.70 5058.30 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90 5289.90
W-5 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 8296.20 8716.80 9030.60 9377.10 9377.10 9846.90 9846.90 10338.60 10338.60 10856.40 10856.40
W-4 4665.90 5018.70 5162.70 5304.60 5548.80 5790.30 6035.10 6402.60 6725.10 7032.00 7283.40 7528.50 7888.20 8183.70 8520.90 8520.90 8691.00 8691.00 8691.00 8691.00 8691.00 8691.00
W-3 4261.20 4438.50 4620.90 4680.30 4870.80 5246.40 5637.30 5821.50 6034.80 6253.80 6648.90 6915.00 7074.30 7243.50 7474.50 7474.50 7474.50 7474.50 7474.50 7474.50 7474.50 7474.50
W-2 3770.40 4127.10 4236.60 4312.20 4556.40 4936.50 5125.20 5310.30 5537.10 5714.40 5874.60 6066.90 6193.20 6293.10 6293.10 6293.10 6293.10 6293.10 6293.10 6293.10 6293.10 6293.10
W-1 3309.30 3666.00 3761.40 3963.90 4203.00 4555.80 4720.20 4950.90 5177.40 5355.60 5519.40 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60 5718.60
E-9 0.00 0.00 0.00 0.00 0.00 0.00 5637.00 5764.80 5925.90 6114.90 6306.60 6612.00 6871.50 7143.30 7560.30 7560.30 7937.70 7937.70 8334.90 8334.90 8752.50 8752.50
E-8 0.00 0.00 0.00 0.00 0.00 4614.60 4818.60 4944.90 5096.10 5260.50 5556.30 5706.30 5961.60 6103.50 6451.80 6451.80 6581.40 6581.40 6581.40 6581.40 6581.40 6581.40
E-7 3207.60 3501.00 3635.40 3812.40 3951.30 4189.50 4323.90 4561.80 4760.10 4895.10 5039.10 5094.90 5282.40 5382.90 5765.40 5765.40 5765.40 5765.40 5765.40 5765.40 5765.40 5765.40
E-6 2774.40 3053.10 3188.10 3318.90 3455.40 3762.60 3882.90 4114.50 4185.30 4236.90 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20 4297.20
E-5 2541.60 2712.90 2844.00 2978.10 3187.20 3405.60 3585.30 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90 3606.90
E-4 2330.40 2449.80 2582.40 2713.50 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00 2829.00
E-3 2103.90 2236.20 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80 2371.80
E-2 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70 2000.70
E-1 >4 Mon 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00 1785.00
E-1 <4 Mon 1650.30 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00"""

In [419]:
milPay = dict()
for line in milPayScaleStr.strip().split("\n")[1:]:
    row = line.split()
    row[0] = row[0].replace('*',"")
    milPay[row[0]] = []
    j = 0
    prev = 0.0
    for val in row[1:]:
        if val == ">4" or val == "<4" or val == "Mon":
            val = row[3]
        fVal = float(val)
        if j == 0:
            milPay[row[0]].append(fVal)
            milPay[row[0]].append(fVal)
            j=1
        elif j < 4:
            milPay[row[0]].append(fVal)
            j+=1
        else:
            milPay[row[0]].append(fVal)
            milPay[row[0]].append(fVal)
            j+=2

### Generate Monthly Income

Fix Date and times

In [420]:
from datetime import timedelta
retirement = basd + timedelta(days=(retirementYears*365))
death = birthday + timedelta(days = (80 *365))

In [421]:
from datetime import datetime as dt

fmt = "%Y%m%d"

basd = dt.strptime(BASD, fmt).replace(day=1)
ets = dt.strptime(ETS, fmt).replace(day=1)
birthday = dt.strptime(BIRTHDAY, fmt).replace(day=1)
dor = {}
for rank, date in DOR:
    dor[dt.strptime(date, fmt).replace(day=1)] = rank


addtlPays = []
for _, amount, start, end, taxable in AdditionalIncomes:
    if end == "":
        addtlPays.append((amount, dt.strptime(start, fmt), retirement, tax))
    else:
        addtlPays.append((amount, dt.strptime(start, fmt), dt.strptime(end, fmt), tax))
        
deduct = []
for _, amount, start, end in deductions:
    if end == "":
        deduct.append((amount, dt.strptime(start, fmt), retirement))
    else:
        deduct.append((amount, dt.strptime(start, fmt), dt.strptime(end, fmt)))
        
bonuss = []
for amount, start, yrs in reenlistmentBonuses:
    bonuss.append((amount, dt.strptime(start, fmt), yrs))
    

Generate Months

In [615]:
from dateutil import rrule
from dateutil.relativedelta import relativedelta

months = {}

curRank = dor[basd]
for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=death):
    yos = relativedelta(month, basd).years
    if month in dor:
        curRank = dor[month]
    if month <= retirement:
        # Base Pay
        income = milPay[curRank][yos]  * .85
        
        # BAH
        mhaCode = mhaCodes[zipcode]
        if dependants:
            income += withDependComb[mhaCode][curRank]
        else:
            income += withoutDependComb[mhaCode][curRank]
        months[month] = income
    else:
        months[month] = 0

Generate High 3

In [616]:
high3Start = retirement + relativedelta(years=-3)
high3Start = high3Start.replace(day=1)

In [617]:
total = 0
for month in rrule.rrule(rrule.MONTHLY, dtstart=high3Start, until=retirement):
    total += months[month]

In [618]:
mul = .025 if BRS else .02
print(mul)

0.02


In [619]:
pension = (total / 36) * (mul * retirementYears)

In [620]:
pensionStart = retirement + relativedelta(months=+1)
pensionStart = pensionStart.replace(day=1)
for month in rrule.rrule(rrule.MONTHLY, dtstart=pensionStart, until=death):
    months[month] += pension

Add additonal duty pays

In [621]:
for amount, start, end, tax in addtlPays:
    for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
        if month >= start and (end == "" or month < end):
            months[month] += amount

Remove deductions

In [622]:
for amount, start, end in deduct:
    for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
        if month >= start and (end == "" or month < end):
            months[month] -= amount

Add Bonus'

In [623]:
print(bonuss)

[(82000, datetime.datetime(2021, 8, 1, 0, 0), 6)]


In [624]:
for amount, date, yrs in bonuss:
    monthly = (amount * .8)/(yrs * 12)
    end = date + relativedelta(months=+(yrs * 12))
    for month in rrule.rrule(rrule.MONTHLY, dtstart=date, until=end):
        months[month] += monthly

## Display Data

### Stay until Retirement

### Get Out at current ETS

In [625]:
yearlyInterstGoal = pension * 12

neededInvestment = yearlyInterstGoal / .08

growthTime = retirement - ets

In [626]:
yearsToGrow = int(abs(growthTime.days / 365)) + 1

In [627]:
yearsToGrow

9

In [628]:
monthlyInvestment = 0
i = 0
while(True):
    total = 0
    for j in range(0, yearsToGrow):
        total = (total + (i * 12)) * 1.08
    if total > neededInvestment:
        monthlyInvestment = i
        break
    i += 1

In [629]:
outMonths = {}
for month in rrule.rrule(rrule.MONTHLY, dtstart=ets, until=retirement):
    outMonths[month] = months[month] + monthlyInvestment

In [632]:
for year  in rrule.rrule(rrule.YEARLY,  dtstart=ets, until=retirement):
    print((outMonths[year] * 12) / .8)

172265.64166666666
172265.64166666666
175153.51666666663
182394.69166666665
184655.26666666663
184655.26666666663
173031.14999999997
184433.47499999995
187826.25
